In [1]:
%run Runge_Adams_FDA.ipynb

In [2]:
t, h, k = symbols(r't, h, k', real=True)
p, q, r = symbols('p, q, r', cls=Function)

## dx xn+1 − xn F (xn+1 ) + F (xn ) = F (x) ⇒ = dt ∆t 2 

Theorem (Cooper, 1987) The implicit midpoint rule automatically inherits each quadratic conservation law. If the field of algebraical integrals of dynamic system is generated by quadratic forms, then the implicit midpoint rule is total conservative. Ref.: Sanz-Serna J.M. // SIAM Review. 2016. Vol. 58, No. 1, pp. 3–33.

In [3]:
s = {
    p(t).diff(t): q(t)*r(t),
    q(t).diff(t): -p(t)*r(t),
    r(t).diff(t): -k**2*p(t)*q(t),
}

In [4]:
(p(t)**2 + q(t)**2).diff(t).subs(s).simplify()

0

In [5]:
(k**2*p(t)**2 + r(t)**2).diff(t).subs(s).simplify()

0

In [6]:
init((p(t), q(t), r(t)), t, h)

In [7]:
set_clip(8, 7, Rational(-7, 1))

In [8]:
expand(T((p(t+h) - 2*p(t) + p(t-h)))/(h*h) + T(p(t)))

117649*h**7*Derivative(p(t), (t, 7))/720 + 117649*h**6*Derivative(p(t), (t, 6))/720 + 16807*h**5*Derivative(p(t), (t, 5))/120 + 17381*h**5*Derivative(p(t), (t, 7))/120 + 2401*h**4*Derivative(p(t), (t, 4))/24 + 36751*h**4*Derivative(p(t), (t, 6))/360 + 343*h**3*Derivative(p(t), (t, 3))/6 + 231*h**3*Derivative(p(t), (t, 5))/4 + 49*h**2*Derivative(p(t), (t, 2))/2 + 295*h**2*Derivative(p(t), (t, 4))/12 + 7*h*Derivative(p(t), t) + 7*h*Derivative(p(t), (t, 3)) + p(t) + Derivative(p(t), (t, 2))

In [9]:
def g(y):
    return Matrix([\
        y[1]*y[2],\
        -y[0]*y[2],\
        -k**2*y[0]*y[1],\
    ])

def f(y):
    return Matrix([\
        clip_n(y[1]*y[2]),\
        clip_n(-y[0]*y[2]),\
        clip_n(-k**2*y[0]*y[1]),\
    ])

s = (('p(t)', 'y[0]'), ('q(t)', 'y[1]'), ('r(t)', 'y[2]'))
def replace(r):
    for a, b in s:
        r = r.replace(a, b)
    return r

In [10]:
start = time.time()

## Gauss–Legendre method

In [11]:
set_clip(8, 7, Rational(0, 1))
m = GaussLegendre2(f, Matrix([p(t), q(t), r(t)]))

In [12]:
F1 = clip((T(p(t+h))-T(p(t)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


p_{tt}/2

h^2 =>


...

h^5 =>


p_{tttttt}/720

h^6 =>


p_{ttttttt}/5040

p_{t} - q r
+h\left(
\frac{p_{tt}}{2}
\right)
+h^2\left(
\frac{p_{ttt}}{6}
\right)
+h^3\left(
\frac{p_{tttt}}{24}
\right)
+h^4\left(
\frac{p_{ttttt}}{120}
\right)
+h^5\left(
\frac{p_{tttttt}}{720}
\right)
+h^6\left(
\frac{p_{ttttttt}}{5040}
\right)


In [13]:
F2 = clip((T(q(t+h))-T(q(t)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


q_{tt}/2

h^2 =>


...

h^5 =>


q_{tttttt}/720

h^6 =>


q_{ttttttt}/5040

p r + q_{t}
+h\left(
\frac{q_{tt}}{2}
\right)
+h^2\left(
\frac{q_{ttt}}{6}
\right)
+h^3\left(
\frac{q_{tttt}}{24}
\right)
+h^4\left(
\frac{q_{ttttt}}{120}
\right)
+h^5\left(
\frac{q_{tttttt}}{720}
\right)
+h^6\left(
\frac{q_{ttttttt}}{5040}
\right)


In [14]:
F3 = clip((T(r(t+h))-T(r(t)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


r_{tt}/2

h^2 =>


...

h^5 =>


r_{tttttt}/720

h^6 =>


r_{ttttttt}/5040

k^{2} p q + r_{t}
+h\left(
\frac{r_{tt}}{2}
\right)
+h^2\left(
\frac{r_{ttt}}{6}
\right)
+h^3\left(
\frac{r_{tttt}}{24}
\right)
+h^4\left(
\frac{r_{ttttt}}{120}
\right)
+h^5\left(
\frac{r_{tttttt}}{720}
\right)
+h^6\left(
\frac{r_{ttttttt}}{5040}
\right)


In [15]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


-k**2*p*q**2/2 - p*r**2/2

h^2 =>


...

h^5 =>


-k**6*p*q**2*(13*p**4 - 38*p**2*q**2 + 10*q**4)/60 - k**4*p*r**2*(13*p**4 - 228*p**2*q**2 + 139*q**4)/60 + k**2*p*r**4*(38*p**2 - 139*q**2)/60 - p*r**6/6

h^6 =>


-k**6*q*r*(151*p**6 - 1303*p**4*q**2 + 1065*p**2*q**4 - 60*q**6)/420 + k**4*q*r**3*(1303*p**4 - 3363*p**2*q**2 + 358*q**4)/420 - k**2*q*r**5*(1065*p**2 - 358*q**2)/420 + q*r**7/7

p_{t} - q r
+h\left(
- \frac{k^{2} p q^{2}}{2} - \frac{p r^{2}}{2}
\right)
+h^2\left(
- \frac{k^{2} q r \left(5 p^{2} - 2 q^{2}\right)}{6} + \frac{q r^{3}}{3}
\right)
+h^3\left(
- \frac{k^{4} p q^{2} \left(4 p^{2} - 3 q^{2}\right)}{12} - \frac{k^{2} p r^{2} \left(2 p^{2} - 9 q^{2}\right)}{6} + \frac{p r^{4}}{4}
\right)
+h^4\left(
- \frac{k^{4} q r \left(8 p^{4} - 27 p^{2} q^{2} + 3 q^{4}\right)}{15} + \frac{k^{2} q r^{3} \left(54 p^{2} - 19 q^{2}\right)}{30} - \frac{q r^{5}}{5}
\right)
+h^5\left(
- \frac{k^{6} p q^{2} \left(13 p^{4} - 38 p^{2} q^{2} + 10 q^{4}\right)}{60} - \frac{k^{4} p r^{2} \left(13 p^{4} - 228 p^{2} q^{2} + 139 q^{4}\right)}{60} + \frac{k^{2} p r^{4} \left(38 p^{2} - 139 q^{2}\right)}{60} - \frac{p r^{6}}{6}
\right)
+h^6\left(
- \frac{k^{6} q r \left(151 p^{6} - 1303 p^{4} q^{2} + 1065 p^{2} q^{4} - 60 q^{6}\right)}{420} + \frac{k^{4} q r^{3} \left(1303 p^{4} - 3363 p^{2} q^{2} + 358 q^{4}\right)}{420} - \frac{k^{2} q r^{5} \left(1065 p^{2} - 358 q^{2}\right)}{420}

In [16]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


k**2*p**2*q/2 - q*r**2/2

h^2 =>


...

h^5 =>


k**6*p**2*q*(10*p**4 - 38*p**2*q**2 + 13*q**4)/60 - k**4*q*r**2*(139*p**4 - 228*p**2*q**2 + 13*q**4)/60 + k**2*q*r**4*(139*p**2 - 38*q**2)/60 - q*r**6/6

h^6 =>


k**6*p*r*(60*p**6 - 1065*p**4*q**2 + 1303*p**2*q**4 - 151*q**6)/420 - k**4*p*r**3*(358*p**4 - 3363*p**2*q**2 + 1303*q**4)/420 + k**2*p*r**5*(358*p**2 - 1065*q**2)/420 - p*r**7/7

p r + q_{t}
+h\left(
\frac{k^{2} p^{2} q}{2} - \frac{q r^{2}}{2}
\right)
+h^2\left(
\frac{k^{2} p r \left(2 p^{2} - 5 q^{2}\right)}{6} - \frac{p r^{3}}{3}
\right)
+h^3\left(
\frac{k^{4} p^{2} q \left(3 p^{2} - 4 q^{2}\right)}{12} - \frac{k^{2} q r^{2} \left(9 p^{2} - 2 q^{2}\right)}{6} + \frac{q r^{4}}{4}
\right)
+h^4\left(
\frac{k^{4} p r \left(3 p^{4} - 27 p^{2} q^{2} + 8 q^{4}\right)}{15} - \frac{k^{2} p r^{3} \left(19 p^{2} - 54 q^{2}\right)}{30} + \frac{p r^{5}}{5}
\right)
+h^5\left(
\frac{k^{6} p^{2} q \left(10 p^{4} - 38 p^{2} q^{2} + 13 q^{4}\right)}{60} - \frac{k^{4} q r^{2} \left(139 p^{4} - 228 p^{2} q^{2} + 13 q^{4}\right)}{60} + \frac{k^{2} q r^{4} \left(139 p^{2} - 38 q^{2}\right)}{60} - \frac{q r^{6}}{6}
\right)
+h^6\left(
\frac{k^{6} p r \left(60 p^{6} - 1065 p^{4} q^{2} + 1303 p^{2} q^{4} - 151 q^{6}\right)}{420} - \frac{k^{4} p r^{3} \left(358 p^{4} - 3363 p^{2} q^{2} + 1303 q^{4}\right)}{420} + \frac{k^{2} p r^{5} \left(358 p^{2} - 1065 q^{2}\right)}{420} - \frac{p r

In [17]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


k**2*r*(p - q)*(p + q)/2

h^2 =>


...

h^5 =>


k**6*r*(p - q)*(p + q)*(10*p**4 - 129*p**2*q**2 + 10*q**4)/60 - 19*k**4*r**3*(p**2 - 2*p*q - q**2)*(p**2 + 2*p*q - q**2)/30 + 13*k**2*r**5*(p - q)*(p + q)/60

h^6 =>


k**8*p*q*(p - q)*(p + q)*(30*p**4 - 149*p**2*q**2 + 30*q**4)/210 - k**6*p*q*r**2*(355*p**4 - 1121*p**2*q**2 + 355*q**4)/140 + 1303*k**4*p*q*r**4*(p - q)*(p + q)/420 - 151*k**2*p*q*r**6/420

k^{2} p q + r_{t}
+h\left(
\frac{k^{2} r \left(p - q\right) \left(p + q\right)}{2}
\right)
+h^2\left(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{3} - \frac{5 k^{2} p q r^{2}}{6}
\right)
+h^3\left(
\frac{k^{4} r \left(p^{2} - 2 p q - q^{2}\right) \left(p^{2} + 2 p q - q^{2}\right)}{4} - \frac{k^{2} r^{3} \left(p - q\right) \left(p + q\right)}{3}
\right)
+h^4\left(
\frac{k^{6} p q \left(6 p^{4} - 19 p^{2} q^{2} + 6 q^{4}\right)}{30} - \frac{9 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{5} + \frac{8 k^{2} p q r^{4}}{15}
\right)
+h^5\left(
\frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(10 p^{4} - 129 p^{2} q^{2} + 10 q^{4}\right)}{60} - \frac{19 k^{4} r^{3} \left(p^{2} - 2 p q - q^{2}\right) \left(p^{2} + 2 p q - q^{2}\right)}{30} + \frac{13 k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{60}
\right)
+h^6\left(
\frac{k^{8} p q \left(p - q\right) \left(p + q\right) \left(30 p^{4} - 149 p^{2} q^{2} + 30 q^{4}\right)}{210} - \frac{k^{6} p q r^{2} \left(355

In [18]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

h^6 =>


(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt} + q*q_{ttttttt} + 7*q_{tttttt}*q_{t} + 21*q_{ttttt}*q_{tt} + 35*q_{tttt}*q_{ttt})/2520

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)
+h^6\left(
\frac{p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt} + q q_{ttttttt} + 7 q_{tttttt} q_{t} + 21 q_{ttttt} q_{tt} + 35 q_{tttt} q_{ttt}}{2520}
\right)


In [19]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


r**2*(p**2 + q**2)

h^2 =>


...

h^5 =>


0

h^6 =>


0

0
+h\left(
r^{2} \left(p^{2} + q^{2}\right)
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
0
\right)
+h^6\left(
0
\right)


In [20]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

h^6 =>


k**2*(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt})/2520 + (r*r_{ttttttt} + 7*r_{tttttt}*r_{t} + 21*r_{ttttt}*r_{tt} + 35*r_{tttt}*r_{ttt})/2520

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)
+h^6\left(
\frac{k^{2} \left(p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt}\right)}{2520} + \frac{r r_{ttttttt} + 7 r_{tttttt} r_{t} + 21 r_{ttttt} r_{tt} + 35 r_{tttt} r_{ttt}}{2520}
\right)


In [21]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


k**4*p**2*q**2 + k**2*q**2*r**2

h^2 =>


...

h^5 =>


0

h^6 =>


0

0
+h\left(
k^{4} p^{2} q^{2} + k^{2} q^{2} r^{2}
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
0
\right)
+h^6\left(
0
\right)


## Crank-Nicolson method

In [22]:
set_clip(5, 4, Rational(0, 1))
m = CrankNicolson2(f, Matrix([p(t), q(t), r(t)]))

In [23]:
F1 = clip((T(p(t+h))-T(p(t)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


p_{tt}/2

h^2 =>


p_{ttt}/6

h^3 =>


p_{tttt}/24

p_{t} - q r
+h\left(
\frac{p_{tt}}{2}
\right)
+h^2\left(
\frac{p_{ttt}}{6}
\right)
+h^3\left(
\frac{p_{tttt}}{24}
\right)


In [24]:
F2 = clip((T(q(t+h))-T(q(t)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


q_{tt}/2

h^2 =>


q_{ttt}/6

h^3 =>


q_{tttt}/24

p r + q_{t}
+h\left(
\frac{q_{tt}}{2}
\right)
+h^2\left(
\frac{q_{ttt}}{6}
\right)
+h^3\left(
\frac{q_{tttt}}{24}
\right)


In [25]:
F3 = clip((T(r(t+h))-T(r(t)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


r_{tt}/2

h^2 =>


r_{ttt}/6

h^3 =>


r_{tttt}/24

k^{2} p q + r_{t}
+h\left(
\frac{r_{tt}}{2}
\right)
+h^2\left(
\frac{r_{ttt}}{6}
\right)
+h^3\left(
\frac{r_{tttt}}{24}
\right)


In [26]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


-k**2*p*q**2/2 - p*r**2/2

h^2 =>


-k**2*q*r*(5*p**2 - 2*q**2)/6 + q*r**3/3

h^3 =>


-k**4*p*q**2*(4*p**2 - 3*q**2)/12 - k**2*p*r**2*(2*p**2 - 9*q**2)/6 + p*r**4/4

p_{t} - q r
+h\left(
- \frac{k^{2} p q^{2}}{2} - \frac{p r^{2}}{2}
\right)
+h^2\left(
- \frac{k^{2} q r \left(5 p^{2} - 2 q^{2}\right)}{6} + \frac{q r^{3}}{3}
\right)
+h^3\left(
- \frac{k^{4} p q^{2} \left(4 p^{2} - 3 q^{2}\right)}{12} - \frac{k^{2} p r^{2} \left(2 p^{2} - 9 q^{2}\right)}{6} + \frac{p r^{4}}{4}
\right)


In [27]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


k**2*p**2*q/2 - q*r**2/2

h^2 =>


k**2*p*r*(2*p**2 - 5*q**2)/6 - p*r**3/3

h^3 =>


k**4*p**2*q*(3*p**2 - 4*q**2)/12 - k**2*q*r**2*(9*p**2 - 2*q**2)/6 + q*r**4/4

p r + q_{t}
+h\left(
\frac{k^{2} p^{2} q}{2} - \frac{q r^{2}}{2}
\right)
+h^2\left(
\frac{k^{2} p r \left(2 p^{2} - 5 q^{2}\right)}{6} - \frac{p r^{3}}{3}
\right)
+h^3\left(
\frac{k^{4} p^{2} q \left(3 p^{2} - 4 q^{2}\right)}{12} - \frac{k^{2} q r^{2} \left(9 p^{2} - 2 q^{2}\right)}{6} + \frac{q r^{4}}{4}
\right)


In [28]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


k**2*r*(p - q)*(p + q)/2

h^2 =>


k**4*p*q*(p - q)*(p + q)/3 - 5*k**2*p*q*r**2/6

h^3 =>


k**4*r*(p**2 - 2*p*q - q**2)*(p**2 + 2*p*q - q**2)/4 - k**2*r**3*(p - q)*(p + q)/3

k^{2} p q + r_{t}
+h\left(
\frac{k^{2} r \left(p - q\right) \left(p + q\right)}{2}
\right)
+h^2\left(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{3} - \frac{5 k^{2} p q r^{2}}{6}
\right)
+h^3\left(
\frac{k^{4} r \left(p^{2} - 2 p q - q^{2}\right) \left(p^{2} + 2 p q - q^{2}\right)}{4} - \frac{k^{2} r^{3} \left(p - q\right) \left(p + q\right)}{3}
\right)


In [29]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


(p*p_{ttt} + 3*p_{tt}*p_{t} + q*q_{ttt} + 3*q_{tt}*q_{t})/3

h^3 =>


(p*p_{tttt} + 4*p_{ttt}*p_{t} + 3*p_{tt}**2 + q*q_{tttt} + 4*q_{ttt}*q_{t} + 3*q_{tt}**2)/12

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)


In [30]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


r**2*(p**2 + q**2)

h^2 =>


0

h^3 =>


0

0
+h\left(
r^{2} \left(p^{2} + q^{2}\right)
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)


In [31]:
replace(str(f4[1].collect(k)))

'y[0]**2*y[2]**2 + y[1]**2*y[2]**2'

In [32]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


k**2*(p*p_{ttt} + 3*p_{tt}*p_{t})/3 + (r*r_{ttt} + 3*r_{tt}*r_{t})/3

h^3 =>


k**2*(p*p_{tttt} + 4*p_{ttt}*p_{t} + 3*p_{tt}**2)/12 + (r*r_{tttt} + 4*r_{ttt}*r_{t} + 3*r_{tt}**2)/12

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)


In [33]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


k**4*p**2*q**2 + k**2*q**2*r**2

h^2 =>


0

h^3 =>


0

0
+h\left(
k^{4} p^{2} q^{2} + k^{2} q^{2} r^{2}
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)


In [34]:
replace(str(f5[1].factor()))

'k**2*(k**2*y[0]**2 + y[2]**2)*y[1]**2'

## "original" Runge–Kutta method

In [35]:
set_clip(7, 6, Rational(0, 1))
m = RungeKutta4(f, Matrix([p(t), q(t), r(t)]))

In [36]:
prn(clip(-m[0]), k)
prnlatex(clip(-m[0]), k)

h^0 =>


-q*r

h^1 =>


k**2*p*q**2/2 + p*r**2/2

h^2 =>


...

h^4 =>


-k**4*p**2*q*r*(7*p**2 - 17*q**2)/48 + k**2*q*r**3*(17*p**2 - 5*q**2)/48

h^5 =>


k**6*p**3*q**2*(2*p**2 - 5*q**2)/96 + k**4*p*r**2*(p**4 - 19*p**2*q**2 + 7*q**4)/48 - k**2*p*r**4*(5*p**2 - 14*q**2)/96

- q r
+h\left(
\frac{k^{2} p q^{2}}{2} + \frac{p r^{2}}{2}
\right)
+h^2\left(
- \frac{k^{2} q r \left(2 p - q\right) \left(2 p + q\right)}{6} + \frac{q r^{3}}{6}
\right)
+h^3\left(
\frac{k^{4} p q^{2} \left(2 p - q\right) \left(2 p + q\right)}{24} + \frac{k^{2} p r^{2} \left(2 p^{2} - 7 q^{2}\right)}{12} - \frac{p r^{4}}{24}
\right)
+h^4\left(
- \frac{k^{4} p^{2} q r \left(7 p^{2} - 17 q^{2}\right)}{48} + \frac{k^{2} q r^{3} \left(17 p^{2} - 5 q^{2}\right)}{48}
\right)
+h^5\left(
\frac{k^{6} p^{3} q^{2} \left(2 p^{2} - 5 q^{2}\right)}{96} + \frac{k^{4} p r^{2} \left(p^{4} - 19 p^{2} q^{2} + 7 q^{4}\right)}{48} - \frac{k^{2} p r^{4} \left(5 p^{2} - 14 q^{2}\right)}{96}
\right)


In [37]:
prn(clip(-m[1]), k)
prnlatex(clip(-m[1]), k)

h^0 =>


p*r

h^1 =>


-k**2*p**2*q/2 + q*r**2/2

h^2 =>


...

h^4 =>


-k**4*p*q**2*r*(17*p**2 - 7*q**2)/48 - k**2*p*r**3*(5*p**2 - 17*q**2)/48

h^5 =>


k**6*p**2*q**3*(5*p**2 - 2*q**2)/96 + k**4*q*r**2*(7*p**4 - 19*p**2*q**2 + q**4)/48 - k**2*q*r**4*(14*p**2 - 5*q**2)/96

p r
+h\left(
- \frac{k^{2} p^{2} q}{2} + \frac{q r^{2}}{2}
\right)
+h^2\left(
\frac{k^{2} p r \left(p - 2 q\right) \left(p + 2 q\right)}{6} - \frac{p r^{3}}{6}
\right)
+h^3\left(
- \frac{k^{4} p^{2} q \left(p - 2 q\right) \left(p + 2 q\right)}{24} + \frac{k^{2} q r^{2} \left(7 p^{2} - 2 q^{2}\right)}{12} - \frac{q r^{4}}{24}
\right)
+h^4\left(
- \frac{k^{4} p q^{2} r \left(17 p^{2} - 7 q^{2}\right)}{48} - \frac{k^{2} p r^{3} \left(5 p^{2} - 17 q^{2}\right)}{48}
\right)
+h^5\left(
\frac{k^{6} p^{2} q^{3} \left(5 p^{2} - 2 q^{2}\right)}{96} + \frac{k^{4} q r^{2} \left(7 p^{4} - 19 p^{2} q^{2} + q^{4}\right)}{48} - \frac{k^{2} q r^{4} \left(14 p^{2} - 5 q^{2}\right)}{96}
\right)


In [38]:
prn(clip(-m[2]), k)
prnlatex(clip(-m[2]), k)

h^0 =>


k**2*p*q

h^1 =>


-k**2*r*(p - q)*(p + q)/2

h^2 =>


...

h^4 =>


-5*k**6*p**3*q**3/48 - 17*k**4*p*q*r**2*(p - q)*(p + q)/48 + 7*k**2*p*q*r**4/48

h^5 =>


7*k**6*p**2*q**2*r*(p - q)*(p + q)/48 + k**4*r**3*(5*p**4 - 38*p**2*q**2 + 5*q**4)/96 - k**2*r**5*(p - q)*(p + q)/48

k^{2} p q
+h\left(
- \frac{k^{2} r \left(p - q\right) \left(p + q\right)}{2}
\right)
+h^2\left(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{6} - \frac{2 k^{2} p q r^{2}}{3}
\right)
+h^3\left(
- \frac{k^{4} r \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{24} + \frac{k^{2} r^{3} \left(p - q\right) \left(p + q\right)}{6}
\right)
+h^4\left(
- \frac{5 k^{6} p^{3} q^{3}}{48} - \frac{17 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{48} + \frac{7 k^{2} p q r^{4}}{48}
\right)
+h^5\left(
\frac{7 k^{6} p^{2} q^{2} r \left(p - q\right) \left(p + q\right)}{48} + \frac{k^{4} r^{3} \left(5 p^{4} - 38 p^{2} q^{2} + 5 q^{4}\right)}{96} - \frac{k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{48}
\right)


In [39]:
F1 = clip((T(p(t+h))-T(p(t)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


k**2*p*q**2/2 + (p*r**2 + p_{tt})/2

h^2 =>


...

h^4 =>


-k**4*p**2*q*r*(7*p**2 - 17*q**2)/48 + k**2*q*r**3*(17*p**2 - 5*q**2)/48 + p_{ttttt}/120

h^5 =>


k**6*p**3*q**2*(2*p**2 - 5*q**2)/96 + k**4*p*r**2*(p**4 - 19*p**2*q**2 + 7*q**4)/48 - k**2*p*r**4*(5*p**2 - 14*q**2)/96 + p_{tttttt}/720

p_{t} - q r
+h\left(
\frac{k^{2} p q^{2}}{2} + \frac{p r^{2} + p_{tt}}{2}
\right)
+h^2\left(
- \frac{k^{2} q r \left(2 p - q\right) \left(2 p + q\right)}{6} + \frac{p_{ttt} + q r^{3}}{6}
\right)
+h^3\left(
\frac{k^{4} p q^{2} \left(2 p - q\right) \left(2 p + q\right)}{24} + \frac{k^{2} p r^{2} \left(2 p^{2} - 7 q^{2}\right)}{12} - \frac{p r^{4} - p_{tttt}}{24}
\right)
+h^4\left(
- \frac{k^{4} p^{2} q r \left(7 p^{2} - 17 q^{2}\right)}{48} + \frac{k^{2} q r^{3} \left(17 p^{2} - 5 q^{2}\right)}{48} + \frac{p_{ttttt}}{120}
\right)
+h^5\left(
\frac{k^{6} p^{3} q^{2} \left(2 p^{2} - 5 q^{2}\right)}{96} + \frac{k^{4} p r^{2} \left(p^{4} - 19 p^{2} q^{2} + 7 q^{4}\right)}{48} - \frac{k^{2} p r^{4} \left(5 p^{2} - 14 q^{2}\right)}{96} + \frac{p_{tttttt}}{720}
\right)


In [40]:
F2 = clip((T(q(t+h))-T(q(t)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


-k**2*p**2*q/2 + (q*r**2 + q_{tt})/2

h^2 =>


...

h^4 =>


-k**4*p*q**2*r*(17*p**2 - 7*q**2)/48 - k**2*p*r**3*(5*p**2 - 17*q**2)/48 + q_{ttttt}/120

h^5 =>


k**6*p**2*q**3*(5*p**2 - 2*q**2)/96 + k**4*q*r**2*(7*p**4 - 19*p**2*q**2 + q**4)/48 - k**2*q*r**4*(14*p**2 - 5*q**2)/96 + q_{tttttt}/720

p r + q_{t}
+h\left(
- \frac{k^{2} p^{2} q}{2} + \frac{q r^{2} + q_{tt}}{2}
\right)
+h^2\left(
\frac{k^{2} p r \left(p - 2 q\right) \left(p + 2 q\right)}{6} - \frac{p r^{3} - q_{ttt}}{6}
\right)
+h^3\left(
- \frac{k^{4} p^{2} q \left(p - 2 q\right) \left(p + 2 q\right)}{24} + \frac{k^{2} q r^{2} \left(7 p^{2} - 2 q^{2}\right)}{12} - \frac{q r^{4} - q_{tttt}}{24}
\right)
+h^4\left(
- \frac{k^{4} p q^{2} r \left(17 p^{2} - 7 q^{2}\right)}{48} - \frac{k^{2} p r^{3} \left(5 p^{2} - 17 q^{2}\right)}{48} + \frac{q_{ttttt}}{120}
\right)
+h^5\left(
\frac{k^{6} p^{2} q^{3} \left(5 p^{2} - 2 q^{2}\right)}{96} + \frac{k^{4} q r^{2} \left(7 p^{4} - 19 p^{2} q^{2} + q^{4}\right)}{48} - \frac{k^{2} q r^{4} \left(14 p^{2} - 5 q^{2}\right)}{96} + \frac{q_{tttttt}}{720}
\right)


In [41]:
F3 = clip((T(r(t+h))-T(r(t)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


-k**2*r*(p - q)*(p + q)/2 + r_{tt}/2

h^2 =>


...

h^4 =>


-5*k**6*p**3*q**3/48 - 17*k**4*p*q*r**2*(p - q)*(p + q)/48 + 7*k**2*p*q*r**4/48 + r_{ttttt}/120

h^5 =>


7*k**6*p**2*q**2*r*(p - q)*(p + q)/48 + k**4*r**3*(5*p**4 - 38*p**2*q**2 + 5*q**4)/96 - k**2*r**5*(p - q)*(p + q)/48 + r_{tttttt}/720

k^{2} p q + r_{t}
+h\left(
- \frac{k^{2} r \left(p - q\right) \left(p + q\right)}{2} + \frac{r_{tt}}{2}
\right)
+h^2\left(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{6} - \frac{2 k^{2} p q r^{2}}{3} + \frac{r_{ttt}}{6}
\right)
+h^3\left(
- \frac{k^{4} r \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{24} + \frac{k^{2} r^{3} \left(p - q\right) \left(p + q\right)}{6} + \frac{r_{tttt}}{24}
\right)
+h^4\left(
- \frac{5 k^{6} p^{3} q^{3}}{48} - \frac{17 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{48} + \frac{7 k^{2} p q r^{4}}{48} + \frac{r_{ttttt}}{120}
\right)
+h^5\left(
\frac{7 k^{6} p^{2} q^{2} r \left(p - q\right) \left(p + q\right)}{48} + \frac{k^{4} r^{3} \left(5 p^{4} - 38 p^{2} q^{2} + 5 q^{4}\right)}{96} - \frac{k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{48} + \frac{r_{tttttt}}{720}
\right)


In [42]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^4 =>


-k**4*q*r*(3*p**4 + 3*p**2*q**2 - 2*q**4)/240 - k**2*q*r**3*(p - q)*(p + q)/80 + q*r**5/120

h^5 =>


-k**6*p*q**2*(p - q)*(p + q)*(p**2 + 2*q**2)/288 - k**4*p*r**2*(p**4 - 10*p**2*q**2 - 3*q**4)/288 - k**2*p*r**4*(p**2 - 3*q**2)/288 + p*r**6/144

p_{t} - q r
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{k^{4} q r \left(3 p^{4} + 3 p^{2} q^{2} - 2 q^{4}\right)}{240} - \frac{k^{2} q r^{3} \left(p - q\right) \left(p + q\right)}{80} + \frac{q r^{5}}{120}
\right)
+h^5\left(
- \frac{k^{6} p q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + 2 q^{2}\right)}{288} - \frac{k^{4} p r^{2} \left(p^{4} - 10 p^{2} q^{2} - 3 q^{4}\right)}{288} - \frac{k^{2} p r^{4} \left(p^{2} - 3 q^{2}\right)}{288} + \frac{p r^{6}}{144}
\right)


In [43]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-k**4*p*r*(2*p**4 - 3*p**2*q**2 - 3*q**4)/240 + k**2*p*r**3*(p - q)*(p + q)/80 - p*r**5/120

h^5 =>


-k**6*p**2*q*(p - q)*(p + q)*(2*p**2 + q**2)/288 + k**4*q*r**2*(3*p**4 + 10*p**2*q**2 - q**4)/288 - k**2*q*r**4*(3*p**2 - q**2)/288 + q*r**6/144

p r + q_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{k^{4} p r \left(2 p^{4} - 3 p^{2} q^{2} - 3 q^{4}\right)}{240} + \frac{k^{2} p r^{3} \left(p - q\right) \left(p + q\right)}{80} - \frac{p r^{5}}{120}
\right)
+h^5\left(
- \frac{k^{6} p^{2} q \left(p - q\right) \left(p + q\right) \left(2 p^{2} + q^{2}\right)}{288} + \frac{k^{4} q r^{2} \left(3 p^{4} + 10 p^{2} q^{2} - q^{4}\right)}{288} - \frac{k^{2} q r^{4} \left(3 p^{2} - q^{2}\right)}{288} + \frac{q r^{6}}{144}
\right)


In [44]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-k**6*p*q*(2*p**4 - 3*p**2*q**2 + 2*q**4)/240 + k**4*p*q*r**2*(p - q)*(p + q)/80 + k**2*p*q*r**4/80

h^5 =>


-k**6*r*(p - q)*(p + q)*(2*p**4 - p**2*q**2 + 2*q**4)/288 + k**4*r**3*(p**4 + 10*p**2*q**2 + q**4)/288 + k**2*r**5*(p - q)*(p + q)/288

k^{2} p q + r_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{k^{6} p q \left(2 p^{4} - 3 p^{2} q^{2} + 2 q^{4}\right)}{240} + \frac{k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{80} + \frac{k^{2} p q r^{4}}{80}
\right)
+h^5\left(
- \frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(2 p^{4} - p^{2} q^{2} + 2 q^{4}\right)}{288} + \frac{k^{4} r^{3} \left(p^{4} + 10 p^{2} q^{2} + q^{4}\right)}{288} + \frac{k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{288}
\right)


In [45]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^4 =>


(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt} + q*q_{ttttt} + 5*q_{tttt}*q_{t} + 10*q_{ttt}*q_{tt})/60

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)


In [46]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


k**4*p*q*r*(p - q)*(p + q)*(p**2 + q**2)/24

h^5 =>


-k**4*r**2*(p - q)**2*(p + q)**2*(p**2 + q**2)/72 + k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/144 - r**6*(p**2 + q**2)/72

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{k^{4} p q r \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{24}
\right)
+h^5\left(
- \frac{k^{4} r^{2} \left(p - q\right)^{2} \left(p + q\right)^{2} \left(p^{2} + q^{2}\right)}{72} + \frac{k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{144} - \frac{r^{6} \left(p^{2} + q^{2}\right)}{72}
\right)


In [47]:
replace(str(f4[4].collect(k)))

'k**4*(y[0]**5*y[1]*y[2]/24 - y[0]*y[1]**5*y[2]/24)'

In [48]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^4 =>


k**2*(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt})/60 + (r*r_{ttttt} + 5*r_{tttt}*r_{t} + 10*r_{ttt}*r_{tt})/60

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)


In [49]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


k**6*p**5*q*r/24 - k**2*p*q*r**5/24

h^5 =>


-k**8*p**2*q**2*(2*p**4 - p**2*q**2 + 2*q**4)/144 + k**6*q**2*r**2*(p - q)*(p + q)*(p**2 + q**2)/72 + k**4*q**2*r**4*(2*p**2 - q**2)/144 - k**2*q**2*r**6/72

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{k^{6} p^{5} q r}{24} - \frac{k^{2} p q r^{5}}{24}
\right)
+h^5\left(
- \frac{k^{8} p^{2} q^{2} \left(2 p^{4} - p^{2} q^{2} + 2 q^{4}\right)}{144} + \frac{k^{6} q^{2} r^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{72} + \frac{k^{4} q^{2} r^{4} \left(2 p^{2} - q^{2}\right)}{144} - \frac{k^{2} q^{2} r^{6}}{72}
\right)


In [50]:
replace(str(f5[4].factor()))

'k**2*(k*y[0] - y[2])*(k*y[0] + y[2])*(k**2*y[0]**2 + y[2]**2)*y[0]*y[1]*y[2]/24'

## Dormand-Prince 5 order

In [51]:
set_clip(8, 7, Rational(0, 1))
m = DormandPrince5(f, Matrix([p(t), q(t), r(t)]))

In [52]:
F1 = clip((T(p(t+h))-T(p(t)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


k**2*p*q**2/2 + (p*r**2 + p_{tt})/2

h^2 =>


...

h^5 =>


k**6*p*q**2*(480*p**4 - 1327*p**2*q**2 + 36*q**4)/21600 + k**4*p*r**2*(720*p**4 - 14783*p**2*q**2 + 6124*q**4)/32400 - k**2*p*r**4*(3981*p**2 - 12248*q**2)/64800 + (6*p*r**6 + 5*p_{tttttt})/3600

h^6 =>


-k**6*p**2*q*r*(321840*p**4 - 10243019*p**2*q**2 + 4313225*q**4)/64152000 + k**4*q*r**3*(30729057*p**4 - 72952646*p**2*q**2 + 4982175*q**4)/192456000 - k**2*q*r**5*(172529*p**2 - 66429*q**2)/2566080 + p_{ttttttt}/5040

p_{t} - q r
+h\left(
\frac{k^{2} p q^{2}}{2} + \frac{p r^{2} + p_{tt}}{2}
\right)
+h^2\left(
- \frac{k^{2} q r \left(2 p - q\right) \left(2 p + q\right)}{6} + \frac{p_{ttt} + q r^{3}}{6}
\right)
+h^3\left(
\frac{k^{4} p q^{2} \left(2 p - q\right) \left(2 p + q\right)}{24} + \frac{k^{2} p r^{2} \left(2 p^{2} - 7 q^{2}\right)}{12} - \frac{p r^{4} - p_{tttt}}{24}
\right)
+h^4\left(
- \frac{k^{4} q r \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{120} + \frac{k^{2} q r^{3} \left(22 p^{2} - 7 q^{2}\right)}{60} - \frac{- p_{ttttt} + q r^{5}}{120}
\right)
+h^5\left(
\frac{k^{6} p q^{2} \left(480 p^{4} - 1327 p^{2} q^{2} + 36 q^{4}\right)}{21600} + \frac{k^{4} p r^{2} \left(720 p^{4} - 14783 p^{2} q^{2} + 6124 q^{4}\right)}{32400} - \frac{k^{2} p r^{4} \left(3981 p^{2} - 12248 q^{2}\right)}{64800} + \frac{6 p r^{6} + 5 p_{tttttt}}{3600}
\right)
+h^6\left(
- \frac{k^{6} p^{2} q r \left(321840 p^{4} - 10243019 p^{2} q^{2} + 4313225 q^{4}\right)}{64152000} + \frac{k^{4

In [53]:
F2 = clip((T(q(t+h))-T(q(t)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


-k**2*p**2*q/2 + (q*r**2 + q_{tt})/2

h^2 =>


...

h^5 =>


-k**6*p**2*q*(36*p**4 - 1327*p**2*q**2 + 480*q**4)/21600 + k**4*q*r**2*(6124*p**4 - 14783*p**2*q**2 + 720*q**4)/32400 - k**2*q*r**4*(12248*p**2 - 3981*q**2)/64800 + (6*q*r**6 + 5*q_{tttttt})/3600

h^6 =>


-k**6*p*q**2*r*(4313225*p**4 - 10243019*p**2*q**2 + 321840*q**4)/64152000 - k**4*p*r**3*(4982175*p**4 - 72952646*p**2*q**2 + 30729057*q**4)/192456000 + k**2*p*r**5*(66429*p**2 - 172529*q**2)/2566080 + q_{ttttttt}/5040

p r + q_{t}
+h\left(
- \frac{k^{2} p^{2} q}{2} + \frac{q r^{2} + q_{tt}}{2}
\right)
+h^2\left(
\frac{k^{2} p r \left(p - 2 q\right) \left(p + 2 q\right)}{6} - \frac{p r^{3} - q_{ttt}}{6}
\right)
+h^3\left(
- \frac{k^{4} p^{2} q \left(p - 2 q\right) \left(p + 2 q\right)}{24} + \frac{k^{2} q r^{2} \left(7 p^{2} - 2 q^{2}\right)}{12} - \frac{q r^{4} - q_{tttt}}{24}
\right)
+h^4\left(
\frac{k^{4} p r \left(p^{2} - 6 p q - 4 q^{2}\right) \left(p^{2} + 6 p q - 4 q^{2}\right)}{120} - \frac{k^{2} p r^{3} \left(7 p^{2} - 22 q^{2}\right)}{60} + \frac{p r^{5} + q_{ttttt}}{120}
\right)
+h^5\left(
- \frac{k^{6} p^{2} q \left(36 p^{4} - 1327 p^{2} q^{2} + 480 q^{4}\right)}{21600} + \frac{k^{4} q r^{2} \left(6124 p^{4} - 14783 p^{2} q^{2} + 720 q^{4}\right)}{32400} - \frac{k^{2} q r^{4} \left(12248 p^{2} - 3981 q^{2}\right)}{64800} + \frac{6 q r^{6} + 5 q_{tttttt}}{3600}
\right)
+h^6\left(
- \frac{k^{6} p q^{2} r \left(4313225 p^{4} - 10243019 p^{2} q^{2} + 321840 q^{4}\right)}{64152000} - \frac{k^{4

In [54]:
F3 = clip((T(r(t+h))-T(r(t)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


-k**2*r*(p - q)*(p + q)/2 + r_{tt}/2

h^2 =>


...

h^5 =>


-k**6*r*(p - q)*(p + q)*(27*p**4 - 3035*p**2*q**2 + 27*q**4)/16200 + k**4*r**3*(3981*p**4 - 29566*p**2*q**2 + 3981*q**4)/64800 - k**2*r**5*(p - q)*(p + q)/45 + r_{tttttt}/720

h^6 =>


-671*k**8*p**3*q**3*(p - q)*(p + q)/25920 - k**6*p*q*r**2*(12939675*p**4 - 72952646*p**2*q**2 + 12939675*q**4)/192456000 + 10243019*k**4*p*q*r**4*(p - q)*(p + q)/64152000 - 149*k**2*p*q*r**6/29700 + r_{ttttttt}/5040

k^{2} p q + r_{t}
+h\left(
- \frac{k^{2} r \left(p - q\right) \left(p + q\right)}{2} + \frac{r_{tt}}{2}
\right)
+h^2\left(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{6} - \frac{2 k^{2} p q r^{2}}{3} + \frac{r_{ttt}}{6}
\right)
+h^3\left(
- \frac{k^{4} r \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{24} + \frac{k^{2} r^{3} \left(p - q\right) \left(p + q\right)}{6} + \frac{r_{tttt}}{24}
\right)
+h^4\left(
\frac{k^{6} p q \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{120} - \frac{11 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{30} + \frac{2 k^{2} p q r^{4}}{15} + \frac{r_{ttttt}}{120}
\right)
+h^5\left(
- \frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(27 p^{4} - 3035 p^{2} q^{2} + 27 q^{4}\right)}{16200} + \frac{k^{4} r^{3} \left(3981 p^{4} - 29566 p^{2} q^{2} + 3981 q^{4}\right)}{64800} - \frac{k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{45} + \frac{r_{tttttt}}{720}
\right)
+h^6\left(
- \frac{671 k^

In [55]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^5 =>


-k**6*p*q**4*(7*p**2 - 6*q**2)/21600 - k**4*p*q**2*r**2*(23*p**2 - 49*q**2)/32400 - 7*k**2*p*r**4*(3*p**2 - 14*q**2)/64800 + p*r**6/3600

h^6 =>


k**6*q*r*(1787130*p**6 - 5290121*p**4*q**2 + 3885725*p**2*q**4 - 106920*q**6)/224532000 - k**4*q*r**3*(15870363*p**4 - 25789604*p**2*q**2 + 1223640*q**4)/673596000 + k**2*q*r**5*(777145*p**2 - 81576*q**2)/44906400 - q*r**7/2100

p_{t} - q r
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{k^{6} p q^{4} \left(7 p^{2} - 6 q^{2}\right)}{21600} - \frac{k^{4} p q^{2} r^{2} \left(23 p^{2} - 49 q^{2}\right)}{32400} - \frac{7 k^{2} p r^{4} \left(3 p^{2} - 14 q^{2}\right)}{64800} + \frac{p r^{6}}{3600}
\right)
+h^6\left(
\frac{k^{6} q r \left(1787130 p^{6} - 5290121 p^{4} q^{2} + 3885725 p^{2} q^{4} - 106920 q^{6}\right)}{224532000} - \frac{k^{4} q r^{3} \left(15870363 p^{4} - 25789604 p^{2} q^{2} + 1223640 q^{4}\right)}{673596000} + \frac{k^{2} q r^{5} \left(777145 p^{2} - 81576 q^{2}\right)}{44906400} - \frac{q r^{7}}{2100}
\right)


In [56]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-k**6*p**4*q*(6*p**2 - 7*q**2)/21600 + k**4*p**2*q*r**2*(49*p**2 - 23*q**2)/32400 - 7*k**2*q*r**4*(14*p**2 - 3*q**2)/64800 + q*r**6/3600

h^6 =>


-k**6*p*r*(106920*p**6 - 3885725*p**4*q**2 + 5290121*p**2*q**4 - 1787130*q**6)/224532000 + k**4*p*r**3*(1223640*p**4 - 25789604*p**2*q**2 + 15870363*q**4)/673596000 - k**2*p*r**5*(81576*p**2 - 777145*q**2)/44906400 + p*r**7/2100

p r + q_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{k^{6} p^{4} q \left(6 p^{2} - 7 q^{2}\right)}{21600} + \frac{k^{4} p^{2} q r^{2} \left(49 p^{2} - 23 q^{2}\right)}{32400} - \frac{7 k^{2} q r^{4} \left(14 p^{2} - 3 q^{2}\right)}{64800} + \frac{q r^{6}}{3600}
\right)
+h^6\left(
- \frac{k^{6} p r \left(106920 p^{6} - 3885725 p^{4} q^{2} + 5290121 p^{2} q^{4} - 1787130 q^{6}\right)}{224532000} + \frac{k^{4} p r^{3} \left(1223640 p^{4} - 25789604 p^{2} q^{2} + 15870363 q^{4}\right)}{673596000} - \frac{k^{2} p r^{5} \left(81576 p^{2} - 777145 q^{2}\right)}{44906400} + \frac{p r^{7}}{2100}
\right)


In [57]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-k**6*r*(p - q)*(p + q)*(9*p**4 - 40*p**2*q**2 + 9*q**4)/32400 + k**4*r**3*(21*p**4 - 46*p**2*q**2 + 21*q**4)/64800

h^6 =>


-k**8*p*q*(p - q)*(p + q)*(27*p**4 - 76*p**2*q**2 + 27*q**4)/56700 + k**6*p*q*r**2*(11657175*p**4 - 25789604*p**2*q**2 + 11657175*q**4)/673596000 - 5290121*k**4*p*q*r**4*(p - q)*(p + q)/224532000 + 6619*k**2*p*q*r**6/831600

k^{2} p q + r_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(9 p^{4} - 40 p^{2} q^{2} + 9 q^{4}\right)}{32400} + \frac{k^{4} r^{3} \left(21 p^{4} - 46 p^{2} q^{2} + 21 q^{4}\right)}{64800}
\right)
+h^6\left(
- \frac{k^{8} p q \left(p - q\right) \left(p + q\right) \left(27 p^{4} - 76 p^{2} q^{2} + 27 q^{4}\right)}{56700} + \frac{k^{6} p q r^{2} \left(11657175 p^{4} - 25789604 p^{2} q^{2} + 11657175 q^{4}\right)}{673596000} - \frac{5290121 k^{4} p q r^{4} \left(p - q\right) \left(p + q\right)}{224532000} + \frac{6619 k^{2} p q r^{6}}{831600}
\right)


In [58]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

h^6 =>


(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt} + q*q_{ttttttt} + 7*q_{tttttt}*q_{t} + 21*q_{ttttt}*q_{tt} + 35*q_{tttt}*q_{ttt})/2520

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)
+h^6\left(
\frac{p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt} + q q_{ttttttt} + 7 q_{tttttt} q_{t} + 21 q_{ttttt} q_{tt} + 35 q_{tttt} q_{ttt}}{2520}
\right)


In [59]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


k**6*p**2*q**2*(p - q)*(p + q)*(p**2 + q**2)/1800 - 13*k**4*p**2*q**2*r**2*(p**2 + q**2)/8100 + 7*k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/10800 - r**6*(p**2 + q**2)/1800

h^6 =>


-k**6*p*q*r*(p**2 + q**2)*(4610*p**4 - 9297*p**2*q**2 + 4610*q**4)/297000 + 702413*k**4*p*q*r**3*(p - q)*(p + q)*(p**2 + q**2)/16038000 - 89863*k**2*p*q*r**5*(p**2 + q**2)/3207600

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{k^{6} p^{2} q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{1800} - \frac{13 k^{4} p^{2} q^{2} r^{2} \left(p^{2} + q^{2}\right)}{8100} + \frac{7 k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{10800} - \frac{r^{6} \left(p^{2} + q^{2}\right)}{1800}
\right)
+h^6\left(
- \frac{k^{6} p q r \left(p^{2} + q^{2}\right) \left(4610 p^{4} - 9297 p^{2} q^{2} + 4610 q^{4}\right)}{297000} + \frac{702413 k^{4} p q r^{3} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{16038000} - \frac{89863 k^{2} p q r^{5} \left(p^{2} + q^{2}\right)}{3207600}
\right)


In [60]:
replace(str(f4[5].collect(k)))

'k**6*(y[0]**6*y[1]**2/1800 - y[0]**2*y[1]**6/1800) + k**4*(-13*y[0]**4*y[1]**2*y[2]**2/8100 - 13*y[0]**2*y[1]**4*y[2]**2/8100) + k**2*(7*y[0]**4*y[2]**4/10800 - 7*y[1]**4*y[2]**4/10800) - y[0]**2*y[2]**6/1800 - y[1]**2*y[2]**6/1800'

In [61]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

h^6 =>


k**2*(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt})/2520 + (r*r_{ttttttt} + 7*r_{tttttt}*r_{t} + 21*r_{ttttt}*r_{tt} + 35*r_{tttt}*r_{ttt})/2520

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)
+h^6\left(
\frac{k^{2} \left(p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt}\right)}{2520} + \frac{r r_{ttttttt} + 7 r_{tttttt} r_{t} + 21 r_{ttttt} r_{tt} + 35 r_{tttt} r_{ttt}}{2520}
\right)


In [62]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


k**8*p**2*q**4*(7*p**2 - 6*q**2)/10800 + k**6*r**2*(p**2 - 3*q**2)*(9*p**4 + p**2*q**2 + 3*q**4)/16200 - k**4*q**2*r**4*(52*p**2 + 21*q**2)/32400 - k**2*p**2*r**6/1800

h^6 =>


-k**8*p**3*q*r*(248940*p**4 - 702413*p**2*q**2 + 449315*q**4)/16038000 + k**6*p*q*r**3*(253098*p**4 - 449315*q**4)/16038000 + k**4*p*q*r**5*(253098*p**2 - 702413*q**2)/16038000 - 461*k**2*p*q*r**7/29700

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{k^{8} p^{2} q^{4} \left(7 p^{2} - 6 q^{2}\right)}{10800} + \frac{k^{6} r^{2} \left(p^{2} - 3 q^{2}\right) \left(9 p^{4} + p^{2} q^{2} + 3 q^{4}\right)}{16200} - \frac{k^{4} q^{2} r^{4} \left(52 p^{2} + 21 q^{2}\right)}{32400} - \frac{k^{2} p^{2} r^{6}}{1800}
\right)
+h^6\left(
- \frac{k^{8} p^{3} q r \left(248940 p^{4} - 702413 p^{2} q^{2} + 449315 q^{4}\right)}{16038000} + \frac{k^{6} p q r^{3} \left(253098 p^{4} - 449315 q^{4}\right)}{16038000} + \frac{k^{4} p q r^{5} \left(253098 p^{2} - 702413 q^{2}\right)}{16038000} - \frac{461 k^{2} p q r^{7}}{29700}
\right)


In [65]:
replace(str(f5[5].factor()))

'k**2*(k**2*y[0]**2 + y[2]**2)*(21*k**4*y[0]**2*y[1]**4 - 18*k**4*y[1]**6 + 18*k**2*y[0]**4*y[2]**2 - 52*k**2*y[0]**2*y[1]**2*y[2]**2 - 21*k**2*y[1]**4*y[2]**2 - 18*y[0]**2*y[2]**4)/32400'

## Dormand-Prince 4 order

In [66]:
set_clip(7, 6, Rational(0, 1))
m = DormandPrince4(f, Matrix([p(t), q(t), r(t)]))

In [67]:
F1 = clip((T(p(t+h))-T(p(t)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


k**2*p*q**2/2 + (p*r**2 + p_{tt})/2

h^2 =>


...

h^4 =>


-k**4*q*r*(431913*p**4 - 1197259*p**2*q**2 + 29619*q**4)/3240000 + k**2*q*r**3*(1197259*p**2 - 385023*q**2)/3240000 - (-1000*p_{ttttt} + 1097*q*r**5)/120000

h^5 =>


k**6*p*q**2*(718336*p**4 - 2000899*p**2*q**2 + 43470*q**4)/32400000 + k**4*p*r**2*(3232512*p**4 - 68009893*p**2*q**2 + 28119005*q**4)/145800000 - k**2*p*r**4*(18008091*p**2 - 56238010*q**2)/291600000 + (483*p*r**6 + 500*p_{tttttt})/360000

p_{t} - q r
+h\left(
\frac{k^{2} p q^{2}}{2} + \frac{p r^{2} + p_{tt}}{2}
\right)
+h^2\left(
- \frac{k^{2} q r \left(2 p - q\right) \left(2 p + q\right)}{6} + \frac{p_{ttt} + q r^{3}}{6}
\right)
+h^3\left(
\frac{k^{4} p q^{2} \left(2 p - q\right) \left(2 p + q\right)}{24} + \frac{k^{2} p r^{2} \left(2 p^{2} - 7 q^{2}\right)}{12} - \frac{p r^{4} - p_{tttt}}{24}
\right)
+h^4\left(
- \frac{k^{4} q r \left(431913 p^{4} - 1197259 p^{2} q^{2} + 29619 q^{4}\right)}{3240000} + \frac{k^{2} q r^{3} \left(1197259 p^{2} - 385023 q^{2}\right)}{3240000} - \frac{- 1000 p_{ttttt} + 1097 q r^{5}}{120000}
\right)
+h^5\left(
\frac{k^{6} p q^{2} \left(718336 p^{4} - 2000899 p^{2} q^{2} + 43470 q^{4}\right)}{32400000} + \frac{k^{4} p r^{2} \left(3232512 p^{4} - 68009893 p^{2} q^{2} + 28119005 q^{4}\right)}{145800000} - \frac{k^{2} p r^{4} \left(18008091 p^{2} - 56238010 q^{2}\right)}{291600000} + \frac{483 p r^{6} + 500 p_{tttttt}}{360000}
\right)


In [68]:
F2 = clip((T(q(t+h))-T(q(t)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


-k**2*p**2*q/2 + (q*r**2 + q_{tt})/2

h^2 =>


...

h^4 =>


k**4*p*r*(29619*p**4 - 1197259*p**2*q**2 + 431913*q**4)/3240000 - k**2*p*r**3*(385023*p**2 - 1197259*q**2)/3240000 + (1097*p*r**5 + 1000*q_{ttttt})/120000

h^5 =>


-k**6*p**2*q*(43470*p**4 - 2000899*p**2*q**2 + 718336*q**4)/32400000 + k**4*q*r**2*(28119005*p**4 - 68009893*p**2*q**2 + 3232512*q**4)/145800000 - k**2*q*r**4*(56238010*p**2 - 18008091*q**2)/291600000 + (483*q*r**6 + 500*q_{tttttt})/360000

p r + q_{t}
+h\left(
- \frac{k^{2} p^{2} q}{2} + \frac{q r^{2} + q_{tt}}{2}
\right)
+h^2\left(
\frac{k^{2} p r \left(p - 2 q\right) \left(p + 2 q\right)}{6} - \frac{p r^{3} - q_{ttt}}{6}
\right)
+h^3\left(
- \frac{k^{4} p^{2} q \left(p - 2 q\right) \left(p + 2 q\right)}{24} + \frac{k^{2} q r^{2} \left(7 p^{2} - 2 q^{2}\right)}{12} - \frac{q r^{4} - q_{tttt}}{24}
\right)
+h^4\left(
\frac{k^{4} p r \left(29619 p^{4} - 1197259 p^{2} q^{2} + 431913 q^{4}\right)}{3240000} - \frac{k^{2} p r^{3} \left(385023 p^{2} - 1197259 q^{2}\right)}{3240000} + \frac{1097 p r^{5} + 1000 q_{ttttt}}{120000}
\right)
+h^5\left(
- \frac{k^{6} p^{2} q \left(43470 p^{4} - 2000899 p^{2} q^{2} + 718336 q^{4}\right)}{32400000} + \frac{k^{4} q r^{2} \left(28119005 p^{4} - 68009893 p^{2} q^{2} + 3232512 q^{4}\right)}{145800000} - \frac{k^{2} q r^{4} \left(56238010 p^{2} - 18008091 q^{2}\right)}{291600000} + \frac{483 q r^{6} + 500 q_{tttttt}}{360000}
\right)


In [69]:
F3 = clip((T(r(t+h))-T(r(t)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


-k**2*r*(p - q)*(p + q)/2 + r_{tt}/2

h^2 =>


...

h^4 =>


k**6*p*q*(9873*p**4 - 128341*p**2*q**2 + 9873*q**4)/1080000 - 1197259*k**4*p*q*r**2*(p - q)*(p + q)/3240000 + 143971*k**2*p*q*r**4/1080000 + r_{ttttt}/120

h^5 =>


-k**6*r*(p - q)*(p + q)*(39123*p**4 - 5584678*p**2*q**2 + 39123*q**4)/29160000 + k**4*r**3*(18008091*p**4 - 136019786*p**2*q**2 + 18008091*q**4)/291600000 - 5612*k**2*r**5*(p - q)*(p + q)/253125 + r_{tttttt}/720

k^{2} p q + r_{t}
+h\left(
- \frac{k^{2} r \left(p - q\right) \left(p + q\right)}{2} + \frac{r_{tt}}{2}
\right)
+h^2\left(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{6} - \frac{2 k^{2} p q r^{2}}{3} + \frac{r_{ttt}}{6}
\right)
+h^3\left(
- \frac{k^{4} r \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{24} + \frac{k^{2} r^{3} \left(p - q\right) \left(p + q\right)}{6} + \frac{r_{tttt}}{24}
\right)
+h^4\left(
\frac{k^{6} p q \left(9873 p^{4} - 128341 p^{2} q^{2} + 9873 q^{4}\right)}{1080000} - \frac{1197259 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{3240000} + \frac{143971 k^{2} p q r^{4}}{1080000} + \frac{r_{ttttt}}{120}
\right)
+h^5\left(
- \frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(39123 p^{4} - 5584678 p^{2} q^{2} + 39123 q^{4}\right)}{29160000} + \frac{k^{4} r^{3} \left(18008091 p^{4} - 136019786 p^{2} q^{2} + 18008091 q^{4}\right)}{291600000} - \frac{5612 k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{253125} + \frac{r

In [70]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^4 =>


k**4*q*r*(87*p**4 + 9259*p**2*q**2 - 2619*q**4)/3240000 + k**2*q*r**3*(9259*p**2 - 7023*q**2)/3240000 - 97*q*r**5/120000

h^5 =>


-k**6*p*q**2*(14324*p**4 - 60511*p**2*q**2 + 27720*q**4)/32400000 - k**4*p*r**2*(64458*p**4 - 68897*p**2*q**2 + 610210*q**4)/145800000 + k**2*p*r**4*(544599*p**2 - 1220420*q**2)/291600000 - 77*p*r**6/90000

p_{t} - q r
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{k^{4} q r \left(87 p^{4} + 9259 p^{2} q^{2} - 2619 q^{4}\right)}{3240000} + \frac{k^{2} q r^{3} \left(9259 p^{2} - 7023 q^{2}\right)}{3240000} - \frac{97 q r^{5}}{120000}
\right)
+h^5\left(
- \frac{k^{6} p q^{2} \left(14324 p^{4} - 60511 p^{2} q^{2} + 27720 q^{4}\right)}{32400000} - \frac{k^{4} p r^{2} \left(64458 p^{4} - 68897 p^{2} q^{2} + 610210 q^{4}\right)}{145800000} + \frac{k^{2} p r^{4} \left(544599 p^{2} - 1220420 q^{2}\right)}{291600000} - \frac{77 p r^{6}}{90000}
\right)


In [71]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


k**4*p*r*(2619*p**4 - 9259*p**2*q**2 - 87*q**4)/3240000 - k**2*p*r**3*(7023*p**2 - 9259*q**2)/3240000 + 97*p*r**5/120000

h^5 =>


k**6*p**2*q*(27720*p**4 - 60511*p**2*q**2 + 14324*q**4)/32400000 - k**4*q*r**2*(610210*p**4 - 68897*p**2*q**2 + 64458*q**4)/145800000 + k**2*q*r**4*(1220420*p**2 - 544599*q**2)/291600000 - 77*q*r**6/90000

p r + q_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{k^{4} p r \left(2619 p^{4} - 9259 p^{2} q^{2} - 87 q^{4}\right)}{3240000} - \frac{k^{2} p r^{3} \left(7023 p^{2} - 9259 q^{2}\right)}{3240000} + \frac{97 p r^{5}}{120000}
\right)
+h^5\left(
\frac{k^{6} p^{2} q \left(27720 p^{4} - 60511 p^{2} q^{2} + 14324 q^{4}\right)}{32400000} - \frac{k^{4} q r^{2} \left(610210 p^{4} - 68897 p^{2} q^{2} + 64458 q^{4}\right)}{145800000} + \frac{k^{2} q r^{4} \left(1220420 p^{2} - 544599 q^{2}\right)}{291600000} - \frac{77 q r^{6}}{90000}
\right)


In [72]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


k**6*p*q*(873*p**4 - 2341*p**2*q**2 + 873*q**4)/1080000 - 9259*k**4*p*q*r**2*(p - q)*(p + q)/3240000 - 29*k**2*p*q*r**4/1080000

h^5 =>


k**6*r*(p - q)*(p + q)*(12474*p**4 - 48547*p**2*q**2 + 12474*q**4)/14580000 - k**4*r**3*(544599*p**4 - 137794*p**2*q**2 + 544599*q**4)/291600000 + 3581*k**2*r**5*(p - q)*(p + q)/8100000

k^{2} p q + r_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{k^{6} p q \left(873 p^{4} - 2341 p^{2} q^{2} + 873 q^{4}\right)}{1080000} - \frac{9259 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{3240000} - \frac{29 k^{2} p q r^{4}}{1080000}
\right)
+h^5\left(
\frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(12474 p^{4} - 48547 p^{2} q^{2} + 12474 q^{4}\right)}{14580000} - \frac{k^{4} r^{3} \left(544599 p^{4} - 137794 p^{2} q^{2} + 544599 q^{4}\right)}{291600000} + \frac{3581 k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{8100000}
\right)


In [73]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^4 =>


(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt} + q*q_{ttttt} + 5*q_{tttt}*q_{t} + 10*q_{ttt}*q_{tt})/60

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)


In [74]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


-451*k**4*p*q*r*(p - q)*(p + q)*(p**2 + q**2)/270000 - 559*k**2*p*q*r**3*(p**2 + q**2)/405000

h^5 =>


67*k**6*p**2*q**2*(p - q)*(p + q)*(p**2 + q**2)/8100000 + k**4*r**2*(p**2 + q**2)*(125343*p**4 + 262475*p**2*q**2 + 125343*q**4)/72900000 - 49331*k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/16200000 + 77*r**6*(p**2 + q**2)/45000

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{451 k^{4} p q r \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{270000} - \frac{559 k^{2} p q r^{3} \left(p^{2} + q^{2}\right)}{405000}
\right)
+h^5\left(
\frac{67 k^{6} p^{2} q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{8100000} + \frac{k^{4} r^{2} \left(p^{2} + q^{2}\right) \left(125343 p^{4} + 262475 p^{2} q^{2} + 125343 q^{4}\right)}{72900000} - \frac{49331 k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{16200000} + \frac{77 r^{6} \left(p^{2} + q^{2}\right)}{45000}
\right)


In [75]:
replace(str(f4[4].collect(k)))

'k**4*(-451*y[0]**5*y[1]*y[2]/270000 + 451*y[0]*y[1]**5*y[2]/270000) + k**2*(-559*y[0]**3*y[1]*y[2]**3/405000 - 559*y[0]*y[1]**3*y[2]**3/405000)'

In [76]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^4 =>


k**2*(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt})/60 + (r*r_{ttttt} + 5*r_{tttt}*r_{t} + 10*r_{ttt}*r_{tt})/60

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)


In [77]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


-k**6*p**3*q*r*(1353*p**2 + 1118*q**2)/810000 - 559*k**4*p*q**3*r**3/405000 + 451*k**2*p*q*r**5/270000

h^5 =>


k**8*p**2*q**2*(27854*p**4 - 49331*p**2*q**2 + 27720*q**4)/16200000 + k**6*r**2*(603*p**6 + 387818*p**4*q**2 + 124740*q**6)/72900000 + k**4*q**2*r**4*(775636*p**2 + 443979*q**2)/145800000 - k**2*r**6*(67*p**2 - 13927*q**2)/8100000

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{k^{6} p^{3} q r \left(1353 p^{2} + 1118 q^{2}\right)}{810000} - \frac{559 k^{4} p q^{3} r^{3}}{405000} + \frac{451 k^{2} p q r^{5}}{270000}
\right)
+h^5\left(
\frac{k^{8} p^{2} q^{2} \left(27854 p^{4} - 49331 p^{2} q^{2} + 27720 q^{4}\right)}{16200000} + \frac{k^{6} r^{2} \left(603 p^{6} + 387818 p^{4} q^{2} + 124740 q^{6}\right)}{72900000} + \frac{k^{4} q^{2} r^{4} \left(775636 p^{2} + 443979 q^{2}\right)}{145800000} - \frac{k^{2} r^{6} \left(67 p^{2} - 13927 q^{2}\right)}{8100000}
\right)


In [78]:
replace(str(f5[4].factor()))

'-k**2*(k**2*y[0]**2 + y[2]**2)*(1353*k**2*y[0]**2 + 1118*k**2*y[1]**2 - 1353*y[2]**2)*y[0]*y[1]*y[2]/810000'

## The implicit midpoint rule for elliptic functions

In [79]:
set_clip(8, 7, Rational(0, 1))

In [82]:
T(p)

ValueError: 
When a single argument is passed to subs it should be a dictionary of
old: new pairs or an iterable of (old, new) tuples.

In [84]:
F1 = clip(4*(T(p(t+h)) - T(p(t)))/h - (T(q(t+h)) + T(q(t)))*(T(r(t+h)) + T(r(t))))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


-4*(-p_{t} + q*r)

h^1 =>


-2*(-p_{tt} + q*r_{t} + q_{t}*r)

h^2 =>


...

h^5 =>


-(-2*p_{tttttt} + 6*q*r_{ttttt} + 6*q_{ttttt}*r + 15*q_{tttt}*r_{t} + 30*q_{ttt}*r_{tt} + 30*q_{tt}*r_{ttt} + 15*q_{t}*r_{tttt})/360

h^6 =>


-(-4*p_{ttttttt} + 14*q*r_{tttttt} + 14*q_{tttttt}*r + 42*q_{ttttt}*r_{t} + 105*q_{tttt}*r_{tt} + 140*q_{ttt}*r_{ttt} + 105*q_{tt}*r_{tttt} + 42*q_{t}*r_{ttttt})/5040

- 4 \left(- p_{t} + q r\right)
+h\left(
- 2 \left(- p_{tt} + q r_{t} + q_{t} r\right)
\right)
+h^2\left(
- \frac{- 2 p_{ttt} + 3 q r_{tt} + 3 q_{tt} r + 3 q_{t} r_{t}}{3}
\right)
+h^3\left(
- \frac{- p_{tttt} + 2 q r_{ttt} + 2 q_{ttt} r + 3 q_{tt} r_{t} + 3 q_{t} r_{tt}}{6}
\right)
+h^4\left(
- \frac{- 2 p_{ttttt} + 5 q r_{tttt} + 5 q_{tttt} r + 10 q_{ttt} r_{t} + 15 q_{tt} r_{tt} + 10 q_{t} r_{ttt}}{60}
\right)
+h^5\left(
- \frac{- 2 p_{tttttt} + 6 q r_{ttttt} + 6 q_{ttttt} r + 15 q_{tttt} r_{t} + 30 q_{ttt} r_{tt} + 30 q_{tt} r_{ttt} + 15 q_{t} r_{tttt}}{360}
\right)
+h^6\left(
- \frac{- 4 p_{ttttttt} + 14 q r_{tttttt} + 14 q_{tttttt} r + 42 q_{ttttt} r_{t} + 105 q_{tttt} r_{tt} + 140 q_{ttt} r_{ttt} + 105 q_{tt} r_{tttt} + 42 q_{t} r_{ttttt}}{5040}
\right)


In [85]:
F2 = clip(4*(T(q(t+h)) - T(q(t)))/h + (T(p(t+h)) + T(p(t)))*(T(r(t+h)) + T(r(t))))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


4*(p*r + q_{t})

h^1 =>


2*(p*r_{t} + p_{t}*r + q_{tt})

h^2 =>


...

h^5 =>


(6*p*r_{ttttt} + 6*p_{ttttt}*r + 15*p_{tttt}*r_{t} + 30*p_{ttt}*r_{tt} + 30*p_{tt}*r_{ttt} + 15*p_{t}*r_{tttt} + 2*q_{tttttt})/360

h^6 =>


(14*p*r_{tttttt} + 14*p_{tttttt}*r + 42*p_{ttttt}*r_{t} + 105*p_{tttt}*r_{tt} + 140*p_{ttt}*r_{ttt} + 105*p_{tt}*r_{tttt} + 42*p_{t}*r_{ttttt} + 4*q_{ttttttt})/5040

4 \left(p r + q_{t}\right)
+h\left(
2 \left(p r_{t} + p_{t} r + q_{tt}\right)
\right)
+h^2\left(
\frac{3 p r_{tt} + 3 p_{tt} r + 3 p_{t} r_{t} + 2 q_{ttt}}{3}
\right)
+h^3\left(
\frac{2 p r_{ttt} + 2 p_{ttt} r + 3 p_{tt} r_{t} + 3 p_{t} r_{tt} + q_{tttt}}{6}
\right)
+h^4\left(
\frac{5 p r_{tttt} + 5 p_{tttt} r + 10 p_{ttt} r_{t} + 15 p_{tt} r_{tt} + 10 p_{t} r_{ttt} + 2 q_{ttttt}}{60}
\right)
+h^5\left(
\frac{6 p r_{ttttt} + 6 p_{ttttt} r + 15 p_{tttt} r_{t} + 30 p_{ttt} r_{tt} + 30 p_{tt} r_{ttt} + 15 p_{t} r_{tttt} + 2 q_{tttttt}}{360}
\right)
+h^6\left(
\frac{14 p r_{tttttt} + 14 p_{tttttt} r + 42 p_{ttttt} r_{t} + 105 p_{tttt} r_{tt} + 140 p_{ttt} r_{ttt} + 105 p_{tt} r_{tttt} + 42 p_{t} r_{ttttt} + 4 q_{ttttttt}}{5040}
\right)


In [86]:
F3 = clip(4*(T(r(t+h)) - T(r(t)))/h + k**2*(T(p(t+h)) + T(p(t)))*(T(q(t+h)) + T(q(t))))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


4*k**2*p*q + 4*r_{t}

h^1 =>


2*k**2*(p*q_{t} + p_{t}*q) + 2*r_{tt}

h^2 =>


...

h^5 =>


k**2*(2*p*q_{ttttt} + 2*p_{ttttt}*q + 5*p_{tttt}*q_{t} + 10*p_{ttt}*q_{tt} + 10*p_{tt}*q_{ttt} + 5*p_{t}*q_{tttt})/120 + r_{tttttt}/180

h^6 =>


k**2*(2*p*q_{tttttt} + 2*p_{tttttt}*q + 6*p_{ttttt}*q_{t} + 15*p_{tttt}*q_{tt} + 20*p_{ttt}*q_{ttt} + 15*p_{tt}*q_{tttt} + 6*p_{t}*q_{ttttt})/720 + r_{ttttttt}/1260

4 k^{2} p q + 4 r_{t}
+h\left(
2 k^{2} \left(p q_{t} + p_{t} q\right) + 2 r_{tt}
\right)
+h^2\left(
k^{2} \left(p q_{tt} + p_{tt} q + p_{t} q_{t}\right) + \frac{2 r_{ttt}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(2 p q_{ttt} + 2 p_{ttt} q + 3 p_{tt} q_{t} + 3 p_{t} q_{tt}\right)}{6} + \frac{r_{tttt}}{6}
\right)
+h^4\left(
\frac{k^{2} \left(p q_{tttt} + p_{tttt} q + 2 p_{ttt} q_{t} + 3 p_{tt} q_{tt} + 2 p_{t} q_{ttt}\right)}{12} + \frac{r_{ttttt}}{30}
\right)
+h^5\left(
\frac{k^{2} \left(2 p q_{ttttt} + 2 p_{ttttt} q + 5 p_{tttt} q_{t} + 10 p_{ttt} q_{tt} + 10 p_{tt} q_{ttt} + 5 p_{t} q_{tttt}\right)}{120} + \frac{r_{tttttt}}{180}
\right)
+h^6\left(
\frac{k^{2} \left(2 p q_{tttttt} + 2 p_{tttttt} q + 6 p_{ttttt} q_{t} + 15 p_{tttt} q_{tt} + 20 p_{ttt} q_{ttt} + 15 p_{tt} q_{tttt} + 6 p_{t} q_{ttttt}\right)}{720} + \frac{r_{ttttttt}}{1260}
\right)


In [87]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


-4*(-p_{t} + q*r)

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


-k**6*q*r*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/1680 + k**4*q*r**3*(29*p**4 + 4*p**2*q**2 + 29*q**4)/840 - 29*k**2*q*r**5*(p - q)*(p + q)/840 + q*r**7/112

- 4 \left(- p_{t} + q r\right)
+h\left(
0
\right)
+h^2\left(
- \frac{k^{2} q r \left(p - q\right) \left(p + q\right)}{3} + \frac{q r^{3}}{3}
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{k^{4} q r \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{60} + \frac{7 k^{2} q r^{3} \left(p - q\right) \left(p + q\right)}{60} - \frac{q r^{5}}{20}
\right)
+h^5\left(
0
\right)
+h^6\left(
- \frac{k^{6} q r \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{1680} + \frac{k^{4} q r^{3} \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{840} - \frac{29 k^{2} q r^{5} \left(p - q\right) \left(p + q\right)}{840} + \frac{q r^{7}}{112}
\right)


In [88]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


4*(p*r + q_{t})

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


k**6*p*r*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/1680 - k**4*p*r**3*(29*p**4 + 4*p**2*q**2 + 29*q**4)/840 + 29*k**2*p*r**5*(p - q)*(p + q)/840 - p*r**7/112

4 \left(p r + q_{t}\right)
+h\left(
0
\right)
+h^2\left(
\frac{k^{2} p r \left(p - q\right) \left(p + q\right)}{3} - \frac{p r^{3}}{3}
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{k^{4} p r \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{60} - \frac{7 k^{2} p r^{3} \left(p - q\right) \left(p + q\right)}{60} + \frac{p r^{5}}{20}
\right)
+h^5\left(
0
\right)
+h^6\left(
\frac{k^{6} p r \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{1680} - \frac{k^{4} p r^{3} \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{840} + \frac{29 k^{2} p r^{5} \left(p - q\right) \left(p + q\right)}{840} - \frac{p r^{7}}{112}
\right)


In [89]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


4*k**2*p*q + 4*r_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


k**8*p*q*(p - q)*(p + q)*(15*p**4 - 43*p**2*q**2 + 15*q**4)/1680 - k**6*p*q*r**2*(29*p**4 + 4*p**2*q**2 + 29*q**4)/840 + 29*k**4*p*q*r**4*(p - q)*(p + q)/840 - k**2*p*q*r**6/112

4 k^{2} p q + 4 r_{t}
+h\left(
0
\right)
+h^2\left(
\frac{k^{4} p q \left(p - q\right) \left(p + q\right)}{3} - \frac{k^{2} p q r^{2}}{3}
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{k^{6} p q \left(3 p^{4} - 7 p^{2} q^{2} + 3 q^{4}\right)}{60} - \frac{7 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{60} + \frac{k^{2} p q r^{4}}{20}
\right)
+h^5\left(
0
\right)
+h^6\left(
\frac{k^{8} p q \left(p - q\right) \left(p + q\right) \left(15 p^{4} - 43 p^{2} q^{2} + 15 q^{4}\right)}{1680} - \frac{k^{6} p q r^{2} \left(29 p^{4} + 4 p^{2} q^{2} + 29 q^{4}\right)}{840} + \frac{29 k^{4} p q r^{4} \left(p - q\right) \left(p + q\right)}{840} - \frac{k^{2} p q r^{6}}{112}
\right)


In [90]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

h^6 =>


(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt} + q*q_{ttttttt} + 7*q_{tttttt}*q_{t} + 21*q_{ttttt}*q_{tt} + 35*q_{tttt}*q_{ttt})/2520

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)
+h^6\left(
\frac{p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt} + q q_{ttttttt} + 7 q_{tttttt} q_{t} + 21 q_{ttttt} q_{tt} + 35 q_{tttt} q_{ttt}}{2520}
\right)


In [91]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


0

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
0
\right)
+h^6\left(
0
\right)


In [92]:
replace(str(f4[1].collect(k)))

'0'

In [93]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

h^6 =>


k**2*(p*p_{ttttttt} + 7*p_{tttttt}*p_{t} + 21*p_{ttttt}*p_{tt} + 35*p_{tttt}*p_{ttt})/2520 + (r*r_{ttttttt} + 7*r_{tttttt}*r_{t} + 21*r_{ttttt}*r_{tt} + 35*r_{tttt}*r_{ttt})/2520

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)
+h^6\left(
\frac{k^{2} \left(p p_{ttttttt} + 7 p_{tttttt} p_{t} + 21 p_{ttttt} p_{tt} + 35 p_{tttt} p_{ttt}\right)}{2520} + \frac{r r_{ttttttt} + 7 r_{tttttt} r_{t} + 21 r_{ttttt} r_{tt} + 35 r_{tttt} r_{ttt}}{2520}
\right)


In [94]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


0

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
0
\right)
+h^6\left(
0
\right)


In [95]:
replace(str(f5[1].factor()))

'0'

### Adams–Bashforth method 4 order

In [96]:
set_clip(7, 6, Rational(0, 1))
m = AdamsBashforth4(g, Matrix([p(t), q(t), r(t)]))

In [97]:
F1 = clip((T(p(t+4*h))-T(p(t+3*h)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


-7*(-p_{tt} + q*r_{t} + q_{t}*r)/2

h^2 =>


...

h^4 =>


-(-4686*p_{ttttt} + 4435*q*r_{tttt} + 4435*q_{tttt}*r + 17740*q_{ttt}*r_{t} + 26610*q_{tt}*r_{tt} + 17740*q_{t}*r_{ttt})/720

h^5 =>


-(-6734*p_{tttttt} + 5757*q*r_{ttttt} + 5757*q_{ttttt}*r + 28785*q_{tttt}*r_{t} + 57570*q_{ttt}*r_{tt} + 57570*q_{tt}*r_{ttt} + 28785*q_{t}*r_{tttt})/1440

p_{t} - q r
+h\left(
- \frac{7 \left(- p_{tt} + q r_{t} + q_{t} r\right)}{2}
\right)
+h^2\left(
- \frac{37 \left(- p_{ttt} + q r_{tt} + q_{tt} r + 2 q_{t} r_{t}\right)}{6}
\right)
+h^3\left(
- \frac{175 \left(- p_{tttt} + q r_{ttt} + q_{ttt} r + 3 q_{tt} r_{t} + 3 q_{t} r_{tt}\right)}{24}
\right)
+h^4\left(
- \frac{- 4686 p_{ttttt} + 4435 q r_{tttt} + 4435 q_{tttt} r + 17740 q_{ttt} r_{t} + 26610 q_{tt} r_{tt} + 17740 q_{t} r_{ttt}}{720}
\right)
+h^5\left(
- \frac{- 6734 p_{tttttt} + 5757 q r_{ttttt} + 5757 q_{ttttt} r + 28785 q_{tttt} r_{t} + 57570 q_{ttt} r_{tt} + 57570 q_{tt} r_{ttt} + 28785 q_{t} r_{tttt}}{1440}
\right)


In [98]:
F2 = clip((T(q(t+4*h))-T(q(t+3*h)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


7*(p*r_{t} + p_{t}*r + q_{tt})/2

h^2 =>


...

h^4 =>


(4435*p*r_{tttt} + 4435*p_{tttt}*r + 17740*p_{ttt}*r_{t} + 26610*p_{tt}*r_{tt} + 17740*p_{t}*r_{ttt} + 4686*q_{ttttt})/720

h^5 =>


(5757*p*r_{ttttt} + 5757*p_{ttttt}*r + 28785*p_{tttt}*r_{t} + 57570*p_{ttt}*r_{tt} + 57570*p_{tt}*r_{ttt} + 28785*p_{t}*r_{tttt} + 6734*q_{tttttt})/1440

p r + q_{t}
+h\left(
\frac{7 \left(p r_{t} + p_{t} r + q_{tt}\right)}{2}
\right)
+h^2\left(
\frac{37 \left(p r_{tt} + p_{tt} r + 2 p_{t} r_{t} + q_{ttt}\right)}{6}
\right)
+h^3\left(
\frac{175 \left(p r_{ttt} + p_{ttt} r + 3 p_{tt} r_{t} + 3 p_{t} r_{tt} + q_{tttt}\right)}{24}
\right)
+h^4\left(
\frac{4435 p r_{tttt} + 4435 p_{tttt} r + 17740 p_{ttt} r_{t} + 26610 p_{tt} r_{tt} + 17740 p_{t} r_{ttt} + 4686 q_{ttttt}}{720}
\right)
+h^5\left(
\frac{5757 p r_{ttttt} + 5757 p_{ttttt} r + 28785 p_{tttt} r_{t} + 57570 p_{ttt} r_{tt} + 57570 p_{tt} r_{ttt} + 28785 p_{t} r_{tttt} + 6734 q_{tttttt}}{1440}
\right)


In [99]:
F3 = clip((T(r(t+4*h))-T(r(t+3*h)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


7*k**2*(p*q_{t} + p_{t}*q)/2 + 7*r_{tt}/2

h^2 =>


...

h^4 =>


887*k**2*(p*q_{tttt} + p_{tttt}*q + 4*p_{ttt}*q_{t} + 6*p_{tt}*q_{tt} + 4*p_{t}*q_{ttt})/144 + 781*r_{ttttt}/120

h^5 =>


1919*k**2*(p*q_{ttttt} + p_{ttttt}*q + 5*p_{tttt}*q_{t} + 10*p_{ttt}*q_{tt} + 10*p_{tt}*q_{ttt} + 5*p_{t}*q_{tttt})/480 + 3367*r_{tttttt}/720

k^{2} p q + r_{t}
+h\left(
\frac{7 k^{2} \left(p q_{t} + p_{t} q\right)}{2} + \frac{7 r_{tt}}{2}
\right)
+h^2\left(
\frac{37 k^{2} \left(p q_{tt} + p_{tt} q + 2 p_{t} q_{t}\right)}{6} + \frac{37 r_{ttt}}{6}
\right)
+h^3\left(
\frac{175 k^{2} \left(p q_{ttt} + p_{ttt} q + 3 p_{tt} q_{t} + 3 p_{t} q_{tt}\right)}{24} + \frac{175 r_{tttt}}{24}
\right)
+h^4\left(
\frac{887 k^{2} \left(p q_{tttt} + p_{tttt} q + 4 p_{ttt} q_{t} + 6 p_{tt} q_{tt} + 4 p_{t} q_{ttt}\right)}{144} + \frac{781 r_{ttttt}}{120}
\right)
+h^5\left(
\frac{1919 k^{2} \left(p q_{ttttt} + p_{ttttt} q + 5 p_{tttt} q_{t} + 10 p_{ttt} q_{tt} + 10 p_{tt} q_{ttt} + 5 p_{t} q_{tttt}\right)}{480} + \frac{3367 r_{tttttt}}{720}
\right)


In [100]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^4 =>


251*k**4*q*r*(4*p**2 - 6*p*q - q**2)*(4*p**2 + 6*p*q - q**2)/720 - 251*k**2*q*r**3*(22*p**2 - 7*q**2)/360 + 251*q*r**5/720

h^5 =>


13*k**6*p*q**2*(4*p**2 - 6*p*q - q**2)*(4*p**2 + 6*p*q - q**2)/24 + 13*k**4*p*r**2*(16*p**4 - 328*p**2*q**2 + 135*q**4)/24 - 13*k**2*p*r**4*(44*p**2 - 135*q**2)/24 + 13*p*r**6/24

p_{t} - q r
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{251 k^{4} q r \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{720} - \frac{251 k^{2} q r^{3} \left(22 p^{2} - 7 q^{2}\right)}{360} + \frac{251 q r^{5}}{720}
\right)
+h^5\left(
\frac{13 k^{6} p q^{2} \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{24} + \frac{13 k^{4} p r^{2} \left(16 p^{4} - 328 p^{2} q^{2} + 135 q^{4}\right)}{24} - \frac{13 k^{2} p r^{4} \left(44 p^{2} - 135 q^{2}\right)}{24} + \frac{13 p r^{6}}{24}
\right)


In [101]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-251*k**4*p*r*(p**2 - 6*p*q - 4*q**2)*(p**2 + 6*p*q - 4*q**2)/720 + 251*k**2*p*r**3*(7*p**2 - 22*q**2)/360 - 251*p*r**5/720

h^5 =>


-13*k**6*p**2*q*(p**2 - 6*p*q - 4*q**2)*(p**2 + 6*p*q - 4*q**2)/24 + 13*k**4*q*r**2*(135*p**4 - 328*p**2*q**2 + 16*q**4)/24 - 13*k**2*q*r**4*(135*p**2 - 44*q**2)/24 + 13*q*r**6/24

p r + q_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{251 k^{4} p r \left(p^{2} - 6 p q - 4 q^{2}\right) \left(p^{2} + 6 p q - 4 q^{2}\right)}{720} + \frac{251 k^{2} p r^{3} \left(7 p^{2} - 22 q^{2}\right)}{360} - \frac{251 p r^{5}}{720}
\right)
+h^5\left(
- \frac{13 k^{6} p^{2} q \left(p^{2} - 6 p q - 4 q^{2}\right) \left(p^{2} + 6 p q - 4 q^{2}\right)}{24} + \frac{13 k^{4} q r^{2} \left(135 p^{4} - 328 p^{2} q^{2} + 16 q^{4}\right)}{24} - \frac{13 k^{2} q r^{4} \left(135 p^{2} - 44 q^{2}\right)}{24} + \frac{13 q r^{6}}{24}
\right)


In [102]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-251*k**6*p*q*(p**2 - 4*p*q + q**2)*(p**2 + 4*p*q + q**2)/720 + 2761*k**4*p*q*r**2*(p - q)*(p + q)/180 - 251*k**2*p*q*r**4/45

h^5 =>


-13*k**6*r*(p - q)*(p + q)*(p**4 - 134*p**2*q**2 + q**4)/24 + 13*k**4*r**3*(11*p**4 - 82*p**2*q**2 + 11*q**4)/6 - 26*k**2*r**5*(p - q)*(p + q)/3

k^{2} p q + r_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{251 k^{6} p q \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{720} + \frac{2761 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{180} - \frac{251 k^{2} p q r^{4}}{45}
\right)
+h^5\left(
- \frac{13 k^{6} r \left(p - q\right) \left(p + q\right) \left(p^{4} - 134 p^{2} q^{2} + q^{4}\right)}{24} + \frac{13 k^{4} r^{3} \left(11 p^{4} - 82 p^{2} q^{2} + 11 q^{4}\right)}{6} - \frac{26 k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{3}
\right)


In [103]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^4 =>


(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt} + q*q_{ttttt} + 5*q_{tttt}*q_{t} + 10*q_{ttt}*q_{tt})/60

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)


In [104]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


-251*k**4*p*q*r*(p - q)*(p + q)*(p**2 + q**2)/24 + 251*k**2*p*q*r**3*(p**2 + q**2)/12

h^5 =>


-529*k**6*p**2*q**2*(p - q)*(p + q)*(p**2 + q**2)/48 - k**4*r**2*(p**2 + q**2)*(581*p**4 - 7856*p**2*q**2 + 581*q**4)/48 + 893*k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/24 - 13*r**6*(p**2 + q**2)/12

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{251 k^{4} p q r \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{24} + \frac{251 k^{2} p q r^{3} \left(p^{2} + q^{2}\right)}{12}
\right)
+h^5\left(
- \frac{529 k^{6} p^{2} q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{48} - \frac{k^{4} r^{2} \left(p^{2} + q^{2}\right) \left(581 p^{4} - 7856 p^{2} q^{2} + 581 q^{4}\right)}{48} + \frac{893 k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{24} - \frac{13 r^{6} \left(p^{2} + q^{2}\right)}{12}
\right)


In [105]:
replace(str(f4[4].collect(k)))

'k**4*(-251*y[0]**5*y[1]*y[2]/24 + 251*y[0]*y[1]**5*y[2]/24) + k**2*(251*y[0]**3*y[1]*y[2]**3/12 + 251*y[0]*y[1]**3*y[2]**3/12)'

In [106]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^4 =>


k**2*(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt})/60 + (r*r_{ttttt} + 5*r_{tttt}*r_{t} + 10*r_{ttt}*r_{tt})/60

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)


In [107]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


-251*k**6*p**3*q*r*(p**2 - 2*q**2)/24 + 251*k**4*p*q**3*r**3/12 + 251*k**2*p*q*r**5/24

h^5 =>


-k**8*p**2*q**2*(581*p**4 - 1786*p**2*q**2 + 52*q**4)/48 - k**6*r**2*(529*p**6 - 7275*p**4*q**2 + 52*q**6)/48 + k**4*q**2*r**4*(7275*p**2 - 1786*q**2)/48 + k**2*r**6*(529*p**2 - 581*q**2)/48

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{251 k^{6} p^{3} q r \left(p^{2} - 2 q^{2}\right)}{24} + \frac{251 k^{4} p q^{3} r^{3}}{12} + \frac{251 k^{2} p q r^{5}}{24}
\right)
+h^5\left(
- \frac{k^{8} p^{2} q^{2} \left(581 p^{4} - 1786 p^{2} q^{2} + 52 q^{4}\right)}{48} - \frac{k^{6} r^{2} \left(529 p^{6} - 7275 p^{4} q^{2} + 52 q^{6}\right)}{48} + \frac{k^{4} q^{2} r^{4} \left(7275 p^{2} - 1786 q^{2}\right)}{48} + \frac{k^{2} r^{6} \left(529 p^{2} - 581 q^{2}\right)}{48}
\right)


In [108]:
replace(str(f5[4].factor()))

'-251*k**2*(k**2*y[0]**2 + y[2]**2)*(k**2*y[0]**2 - 2*k**2*y[1]**2 - y[2]**2)*y[0]*y[1]*y[2]/24'

### Adams–Moulton method 4 order

In [109]:
set_clip(7, 6, Rational(0, 1))
m = AdamsMoulton4(g, Matrix([p(t), q(t), r(t)]))

In [110]:
F1 = clip((T(p(t+3*h))-T(p(t+2*h)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


-5*(-p_{tt} + q*r_{t} + q_{t}*r)/2

h^2 =>


...

h^4 =>


-(-1266*p_{ttttt} + 1285*q*r_{tttt} + 1285*q_{tttt}*r + 5140*q_{ttt}*r_{t} + 7710*q_{tt}*r_{tt} + 5140*q_{t}*r_{ttt})/720

h^5 =>


-(-266*p_{tttttt} + 279*q*r_{ttttt} + 279*q_{ttttt}*r + 1395*q_{tttt}*r_{t} + 2790*q_{ttt}*r_{tt} + 2790*q_{tt}*r_{ttt} + 1395*q_{t}*r_{tttt})/288

p_{t} - q r
+h\left(
- \frac{5 \left(- p_{tt} + q r_{t} + q_{t} r\right)}{2}
\right)
+h^2\left(
- \frac{19 \left(- p_{ttt} + q r_{tt} + q_{tt} r + 2 q_{t} r_{t}\right)}{6}
\right)
+h^3\left(
- \frac{65 \left(- p_{tttt} + q r_{ttt} + q_{ttt} r + 3 q_{tt} r_{t} + 3 q_{t} r_{tt}\right)}{24}
\right)
+h^4\left(
- \frac{- 1266 p_{ttttt} + 1285 q r_{tttt} + 1285 q_{tttt} r + 5140 q_{ttt} r_{t} + 7710 q_{tt} r_{tt} + 5140 q_{t} r_{ttt}}{720}
\right)
+h^5\left(
- \frac{- 266 p_{tttttt} + 279 q r_{ttttt} + 279 q_{ttttt} r + 1395 q_{tttt} r_{t} + 2790 q_{ttt} r_{tt} + 2790 q_{tt} r_{ttt} + 1395 q_{t} r_{tttt}}{288}
\right)


In [111]:
F2 = clip((T(q(t+3*h))-T(q(t+2*h)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


5*(p*r_{t} + p_{t}*r + q_{tt})/2

h^2 =>


...

h^4 =>


(1285*p*r_{tttt} + 1285*p_{tttt}*r + 5140*p_{ttt}*r_{t} + 7710*p_{tt}*r_{tt} + 5140*p_{t}*r_{ttt} + 1266*q_{ttttt})/720

h^5 =>


(279*p*r_{ttttt} + 279*p_{ttttt}*r + 1395*p_{tttt}*r_{t} + 2790*p_{ttt}*r_{tt} + 2790*p_{tt}*r_{ttt} + 1395*p_{t}*r_{tttt} + 266*q_{tttttt})/288

p r + q_{t}
+h\left(
\frac{5 \left(p r_{t} + p_{t} r + q_{tt}\right)}{2}
\right)
+h^2\left(
\frac{19 \left(p r_{tt} + p_{tt} r + 2 p_{t} r_{t} + q_{ttt}\right)}{6}
\right)
+h^3\left(
\frac{65 \left(p r_{ttt} + p_{ttt} r + 3 p_{tt} r_{t} + 3 p_{t} r_{tt} + q_{tttt}\right)}{24}
\right)
+h^4\left(
\frac{1285 p r_{tttt} + 1285 p_{tttt} r + 5140 p_{ttt} r_{t} + 7710 p_{tt} r_{tt} + 5140 p_{t} r_{ttt} + 1266 q_{ttttt}}{720}
\right)
+h^5\left(
\frac{279 p r_{ttttt} + 279 p_{ttttt} r + 1395 p_{tttt} r_{t} + 2790 p_{ttt} r_{tt} + 2790 p_{tt} r_{ttt} + 1395 p_{t} r_{tttt} + 266 q_{tttttt}}{288}
\right)


In [112]:
F3 = clip((T(r(t+3*h))-T(r(t+2*h)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


5*k**2*(p*q_{t} + p_{t}*q)/2 + 5*r_{tt}/2

h^2 =>


...

h^4 =>


257*k**2*(p*q_{tttt} + p_{tttt}*q + 4*p_{ttt}*q_{t} + 6*p_{tt}*q_{tt} + 4*p_{t}*q_{ttt})/144 + 211*r_{ttttt}/120

h^5 =>


31*k**2*(p*q_{ttttt} + p_{ttttt}*q + 5*p_{tttt}*q_{t} + 10*p_{ttt}*q_{tt} + 10*p_{tt}*q_{ttt} + 5*p_{t}*q_{tttt})/32 + 133*r_{tttttt}/144

k^{2} p q + r_{t}
+h\left(
\frac{5 k^{2} \left(p q_{t} + p_{t} q\right)}{2} + \frac{5 r_{tt}}{2}
\right)
+h^2\left(
\frac{19 k^{2} \left(p q_{tt} + p_{tt} q + 2 p_{t} q_{t}\right)}{6} + \frac{19 r_{ttt}}{6}
\right)
+h^3\left(
\frac{65 k^{2} \left(p q_{ttt} + p_{ttt} q + 3 p_{tt} q_{t} + 3 p_{t} q_{tt}\right)}{24} + \frac{65 r_{tttt}}{24}
\right)
+h^4\left(
\frac{257 k^{2} \left(p q_{tttt} + p_{tttt} q + 4 p_{ttt} q_{t} + 6 p_{tt} q_{tt} + 4 p_{t} q_{ttt}\right)}{144} + \frac{211 r_{ttttt}}{120}
\right)
+h^5\left(
\frac{31 k^{2} \left(p q_{ttttt} + p_{ttttt} q + 5 p_{tttt} q_{t} + 10 p_{ttt} q_{tt} + 10 p_{tt} q_{ttt} + 5 p_{t} q_{tttt}\right)}{32} + \frac{133 r_{tttttt}}{144}
\right)


In [113]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^4 =>


-19*k**4*q*r*(4*p**2 - 6*p*q - q**2)*(4*p**2 + 6*p*q - q**2)/720 + 19*k**2*q*r**3*(22*p**2 - 7*q**2)/360 - 19*q*r**5/720

h^5 =>


-k**6*p*q**2*(4*p**2 - 6*p*q - q**2)*(4*p**2 + 6*p*q - q**2)/48 - k**4*p*r**2*(16*p**4 - 328*p**2*q**2 + 135*q**4)/48 + k**2*p*r**4*(44*p**2 - 135*q**2)/48 - p*r**6/48

p_{t} - q r
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
- \frac{19 k^{4} q r \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{720} + \frac{19 k^{2} q r^{3} \left(22 p^{2} - 7 q^{2}\right)}{360} - \frac{19 q r^{5}}{720}
\right)
+h^5\left(
- \frac{k^{6} p q^{2} \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{48} - \frac{k^{4} p r^{2} \left(16 p^{4} - 328 p^{2} q^{2} + 135 q^{4}\right)}{48} + \frac{k^{2} p r^{4} \left(44 p^{2} - 135 q^{2}\right)}{48} - \frac{p r^{6}}{48}
\right)


In [114]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


19*k**4*p*r*(p**2 - 6*p*q - 4*q**2)*(p**2 + 6*p*q - 4*q**2)/720 - 19*k**2*p*r**3*(7*p**2 - 22*q**2)/360 + 19*p*r**5/720

h^5 =>


k**6*p**2*q*(p**2 - 6*p*q - 4*q**2)*(p**2 + 6*p*q - 4*q**2)/48 - k**4*q*r**2*(135*p**4 - 328*p**2*q**2 + 16*q**4)/48 + k**2*q*r**4*(135*p**2 - 44*q**2)/48 - q*r**6/48

p r + q_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{19 k^{4} p r \left(p^{2} - 6 p q - 4 q^{2}\right) \left(p^{2} + 6 p q - 4 q^{2}\right)}{720} - \frac{19 k^{2} p r^{3} \left(7 p^{2} - 22 q^{2}\right)}{360} + \frac{19 p r^{5}}{720}
\right)
+h^5\left(
\frac{k^{6} p^{2} q \left(p^{2} - 6 p q - 4 q^{2}\right) \left(p^{2} + 6 p q - 4 q^{2}\right)}{48} - \frac{k^{4} q r^{2} \left(135 p^{4} - 328 p^{2} q^{2} + 16 q^{4}\right)}{48} + \frac{k^{2} q r^{4} \left(135 p^{2} - 44 q^{2}\right)}{48} - \frac{q r^{6}}{48}
\right)


In [115]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


19*k**6*p*q*(p**2 - 4*p*q + q**2)*(p**2 + 4*p*q + q**2)/720 - 209*k**4*p*q*r**2*(p - q)*(p + q)/180 + 19*k**2*p*q*r**4/45

h^5 =>


k**6*r*(p - q)*(p + q)*(p**4 - 134*p**2*q**2 + q**4)/48 - k**4*r**3*(11*p**4 - 82*p**2*q**2 + 11*q**4)/12 + k**2*r**5*(p - q)*(p + q)/3

k^{2} p q + r_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{19 k^{6} p q \left(p^{2} - 4 p q + q^{2}\right) \left(p^{2} + 4 p q + q^{2}\right)}{720} - \frac{209 k^{4} p q r^{2} \left(p - q\right) \left(p + q\right)}{180} + \frac{19 k^{2} p q r^{4}}{45}
\right)
+h^5\left(
\frac{k^{6} r \left(p - q\right) \left(p + q\right) \left(p^{4} - 134 p^{2} q^{2} + q^{4}\right)}{48} - \frac{k^{4} r^{3} \left(11 p^{4} - 82 p^{2} q^{2} + 11 q^{4}\right)}{12} + \frac{k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{3}
\right)


In [116]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^4 =>


(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt} + q*q_{ttttt} + 5*q_{tttt}*q_{t} + 10*q_{ttt}*q_{tt})/60

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)


In [117]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


19*k**4*p*q*r*(p - q)*(p + q)*(p**2 + q**2)/24 - 19*k**2*p*q*r**3*(p**2 + q**2)/12

h^5 =>


11*k**6*p**2*q**2*(p - q)*(p + q)*(p**2 + q**2)/48 + k**4*r**2*(p**2 + q**2)*(13*p**4 - 190*p**2*q**2 + 13*q**4)/48 - 25*k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/24 + r**6*(p**2 + q**2)/24

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{19 k^{4} p q r \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{24} - \frac{19 k^{2} p q r^{3} \left(p^{2} + q^{2}\right)}{12}
\right)
+h^5\left(
\frac{11 k^{6} p^{2} q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{48} + \frac{k^{4} r^{2} \left(p^{2} + q^{2}\right) \left(13 p^{4} - 190 p^{2} q^{2} + 13 q^{4}\right)}{48} - \frac{25 k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{24} + \frac{r^{6} \left(p^{2} + q^{2}\right)}{24}
\right)


In [118]:
replace(str(f4[4].collect(k)))

'k**4*(19*y[0]**5*y[1]*y[2]/24 - 19*y[0]*y[1]**5*y[2]/24) + k**2*(-19*y[0]**3*y[1]*y[2]**3/12 - 19*y[0]*y[1]**3*y[2]**3/12)'

In [119]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^4 =>


k**2*(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt})/60 + (r*r_{ttttt} + 5*r_{tttt}*r_{t} + 10*r_{ttt}*r_{tt})/60

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)


In [120]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


19*k**6*p**3*q*r*(p**2 - 2*q**2)/24 - 19*k**4*p*q**3*r**3/12 - 19*k**2*p*q*r**5/24

h^5 =>


k**8*p**2*q**2*(13*p**4 - 50*p**2*q**2 + 2*q**4)/48 + k**6*r**2*(11*p**6 - 177*p**4*q**2 + 2*q**6)/48 - k**4*q**2*r**4*(177*p**2 - 50*q**2)/48 - k**2*r**6*(11*p**2 - 13*q**2)/48

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
\frac{19 k^{6} p^{3} q r \left(p^{2} - 2 q^{2}\right)}{24} - \frac{19 k^{4} p q^{3} r^{3}}{12} - \frac{19 k^{2} p q r^{5}}{24}
\right)
+h^5\left(
\frac{k^{8} p^{2} q^{2} \left(13 p^{4} - 50 p^{2} q^{2} + 2 q^{4}\right)}{48} + \frac{k^{6} r^{2} \left(11 p^{6} - 177 p^{4} q^{2} + 2 q^{6}\right)}{48} - \frac{k^{4} q^{2} r^{4} \left(177 p^{2} - 50 q^{2}\right)}{48} - \frac{k^{2} r^{6} \left(11 p^{2} - 13 q^{2}\right)}{48}
\right)


In [127]:
replace(str(f5[4].factor()))

'19*k**2*(k**2*y[0]**2 + y[2]**2)*(k**2*y[0]**2 - 2*k**2*y[1]**2 - y[2]**2)*y[0]*y[1]*y[2]/24'

### Adams–Bashforth method 5 order

In [128]:
set_clip(7, 6, Rational(0, 1))
m = AdamsBashforth5(g, Matrix([p(t), q(t), r(t)]))

In [129]:
F1 = clip((T(p(t+5*h))-T(p(t+4*h)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


-9*(-p_{tt} + q*r_{t} + q_{t}*r)/2

h^2 =>


...

h^4 =>


-2101*(-p_{ttttt} + q*r_{tttt} + q_{tttt}*r + 4*q_{ttt}*r_{t} + 6*q_{tt}*r_{tt} + 4*q_{t}*r_{ttt})/120

h^5 =>


-(-23058*p_{tttttt} + 22583*q*r_{ttttt} + 22583*q_{ttttt}*r + 112915*q_{tttt}*r_{t} + 225830*q_{ttt}*r_{tt} + 225830*q_{tt}*r_{ttt} + 112915*q_{t}*r_{tttt})/1440

p_{t} - q r
+h\left(
- \frac{9 \left(- p_{tt} + q r_{t} + q_{t} r\right)}{2}
\right)
+h^2\left(
- \frac{61 \left(- p_{ttt} + q r_{tt} + q_{tt} r + 2 q_{t} r_{t}\right)}{6}
\right)
+h^3\left(
- \frac{123 \left(- p_{tttt} + q r_{ttt} + q_{ttt} r + 3 q_{tt} r_{t} + 3 q_{t} r_{tt}\right)}{8}
\right)
+h^4\left(
- \frac{2101 \left(- p_{ttttt} + q r_{tttt} + q_{tttt} r + 4 q_{ttt} r_{t} + 6 q_{tt} r_{tt} + 4 q_{t} r_{ttt}\right)}{120}
\right)
+h^5\left(
- \frac{- 23058 p_{tttttt} + 22583 q r_{ttttt} + 22583 q_{ttttt} r + 112915 q_{tttt} r_{t} + 225830 q_{ttt} r_{tt} + 225830 q_{tt} r_{ttt} + 112915 q_{t} r_{tttt}}{1440}
\right)


In [130]:
F2 = clip((T(q(t+5*h))-T(q(t+4*h)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


9*(p*r_{t} + p_{t}*r + q_{tt})/2

h^2 =>


...

h^4 =>


2101*(p*r_{tttt} + p_{tttt}*r + 4*p_{ttt}*r_{t} + 6*p_{tt}*r_{tt} + 4*p_{t}*r_{ttt} + q_{ttttt})/120

h^5 =>


(22583*p*r_{ttttt} + 22583*p_{ttttt}*r + 112915*p_{tttt}*r_{t} + 225830*p_{ttt}*r_{tt} + 225830*p_{tt}*r_{ttt} + 112915*p_{t}*r_{tttt} + 23058*q_{tttttt})/1440

p r + q_{t}
+h\left(
\frac{9 \left(p r_{t} + p_{t} r + q_{tt}\right)}{2}
\right)
+h^2\left(
\frac{61 \left(p r_{tt} + p_{tt} r + 2 p_{t} r_{t} + q_{ttt}\right)}{6}
\right)
+h^3\left(
\frac{123 \left(p r_{ttt} + p_{ttt} r + 3 p_{tt} r_{t} + 3 p_{t} r_{tt} + q_{tttt}\right)}{8}
\right)
+h^4\left(
\frac{2101 \left(p r_{tttt} + p_{tttt} r + 4 p_{ttt} r_{t} + 6 p_{tt} r_{tt} + 4 p_{t} r_{ttt} + q_{ttttt}\right)}{120}
\right)
+h^5\left(
\frac{22583 p r_{ttttt} + 22583 p_{ttttt} r + 112915 p_{tttt} r_{t} + 225830 p_{ttt} r_{tt} + 225830 p_{tt} r_{ttt} + 112915 p_{t} r_{tttt} + 23058 q_{tttttt}}{1440}
\right)


In [131]:
F3 = clip((T(r(t+5*h))-T(r(t+4*h)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


9*k**2*(p*q_{t} + p_{t}*q)/2 + 9*r_{tt}/2

h^2 =>


...

h^4 =>


2101*k**2*(p*q_{tttt} + p_{tttt}*q + 4*p_{ttt}*q_{t} + 6*p_{tt}*q_{tt} + 4*p_{t}*q_{ttt})/120 + 2101*r_{ttttt}/120

h^5 =>


22583*k**2*(p*q_{ttttt} + p_{ttttt}*q + 5*p_{tttt}*q_{t} + 10*p_{ttt}*q_{tt} + 10*p_{tt}*q_{ttt} + 5*p_{t}*q_{tttt})/1440 + 1281*r_{tttttt}/80

k^{2} p q + r_{t}
+h\left(
\frac{9 k^{2} \left(p q_{t} + p_{t} q\right)}{2} + \frac{9 r_{tt}}{2}
\right)
+h^2\left(
\frac{61 k^{2} \left(p q_{tt} + p_{tt} q + 2 p_{t} q_{t}\right)}{6} + \frac{61 r_{ttt}}{6}
\right)
+h^3\left(
\frac{123 k^{2} \left(p q_{ttt} + p_{ttt} q + 3 p_{tt} q_{t} + 3 p_{t} q_{tt}\right)}{8} + \frac{123 r_{tttt}}{8}
\right)
+h^4\left(
\frac{2101 k^{2} \left(p q_{tttt} + p_{tttt} q + 4 p_{ttt} q_{t} + 6 p_{tt} q_{tt} + 4 p_{t} q_{ttt}\right)}{120} + \frac{2101 r_{ttttt}}{120}
\right)
+h^5\left(
\frac{22583 k^{2} \left(p q_{ttttt} + p_{ttttt} q + 5 p_{tttt} q_{t} + 10 p_{ttt} q_{tt} + 10 p_{tt} q_{ttt} + 5 p_{t} q_{tttt}\right)}{1440} + \frac{1281 r_{tttttt}}{80}
\right)


In [132]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-95*k**6*p*q**2*(4*p**2 - 6*p*q - q**2)*(4*p**2 + 6*p*q - q**2)/288 - 95*k**4*p*r**2*(16*p**4 - 328*p**2*q**2 + 135*q**4)/288 + 95*k**2*p*r**4*(44*p**2 - 135*q**2)/288 - 95*p*r**6/288

p_{t} - q r
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{95 k^{6} p q^{2} \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{288} - \frac{95 k^{4} p r^{2} \left(16 p^{4} - 328 p^{2} q^{2} + 135 q^{4}\right)}{288} + \frac{95 k^{2} p r^{4} \left(44 p^{2} - 135 q^{2}\right)}{288} - \frac{95 p r^{6}}{288}
\right)


In [133]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


95*k**6*p**2*q*(p**2 - 6*p*q - 4*q**2)*(p**2 + 6*p*q - 4*q**2)/288 - 95*k**4*q*r**2*(135*p**4 - 328*p**2*q**2 + 16*q**4)/288 + 95*k**2*q*r**4*(135*p**2 - 44*q**2)/288 - 95*q*r**6/288

p r + q_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{95 k^{6} p^{2} q \left(p^{2} - 6 p q - 4 q^{2}\right) \left(p^{2} + 6 p q - 4 q^{2}\right)}{288} - \frac{95 k^{4} q r^{2} \left(135 p^{4} - 328 p^{2} q^{2} + 16 q^{4}\right)}{288} + \frac{95 k^{2} q r^{4} \left(135 p^{2} - 44 q^{2}\right)}{288} - \frac{95 q r^{6}}{288}
\right)


In [134]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


95*k**6*r*(p - q)*(p + q)*(p**4 - 134*p**2*q**2 + q**4)/288 - 95*k**4*r**3*(11*p**4 - 82*p**2*q**2 + 11*q**4)/72 + 95*k**2*r**5*(p - q)*(p + q)/18

k^{2} p q + r_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{95 k^{6} r \left(p - q\right) \left(p + q\right) \left(p^{4} - 134 p^{2} q^{2} + q^{4}\right)}{288} - \frac{95 k^{4} r^{3} \left(11 p^{4} - 82 p^{2} q^{2} + 11 q^{4}\right)}{72} + \frac{95 k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{18}
\right)


In [135]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^4 =>


(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt} + q*q_{ttttt} + 5*q_{tttt}*q_{t} + 10*q_{ttt}*q_{tt})/60

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)


In [136]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


475*k**6*p**2*q**2*(p - q)*(p + q)*(p**2 + q**2)/48 + 95*k**4*r**2*(p**2 + q**2)*(16*p**4 - 209*p**2*q**2 + 16*q**4)/144 - 1045*k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/36 + 95*r**6*(p**2 + q**2)/144

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{475 k^{6} p^{2} q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{48} + \frac{95 k^{4} r^{2} \left(p^{2} + q^{2}\right) \left(16 p^{4} - 209 p^{2} q^{2} + 16 q^{4}\right)}{144} - \frac{1045 k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{36} + \frac{95 r^{6} \left(p^{2} + q^{2}\right)}{144}
\right)


In [137]:
replace(str(f4[5].collect(k)))

'k**6*(475*y[0]**6*y[1]**2/48 - 475*y[0]**2*y[1]**6/48) + k**4*(95*y[0]**6*y[2]**2/9 - 18335*y[0]**4*y[1]**2*y[2]**2/144 - 18335*y[0]**2*y[1]**4*y[2]**2/144 + 95*y[1]**6*y[2]**2/9) + k**2*(-1045*y[0]**4*y[2]**4/36 + 1045*y[1]**4*y[2]**4/36) + 95*y[0]**2*y[2]**6/144 + 95*y[1]**2*y[2]**6/144'

In [138]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^4 =>


k**2*(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt})/60 + (r*r_{ttttt} + 5*r_{tttt}*r_{t} + 10*r_{ttt}*r_{tt})/60

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)


In [139]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


95*k**8*p**2*q**2*(4*p**2 - 6*p*q - q**2)*(4*p**2 + 6*p*q - q**2)/144 + 95*k**6*r**2*(15*p**6 - 193*p**4*q**2 + q**6)/144 - 95*k**4*q**2*r**4*(193*p**2 - 44*q**2)/144 - 95*k**2*r**6*(15*p**2 - 16*q**2)/144

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{95 k^{8} p^{2} q^{2} \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{144} + \frac{95 k^{6} r^{2} \left(15 p^{6} - 193 p^{4} q^{2} + q^{6}\right)}{144} - \frac{95 k^{4} q^{2} r^{4} \left(193 p^{2} - 44 q^{2}\right)}{144} - \frac{95 k^{2} r^{6} \left(15 p^{2} - 16 q^{2}\right)}{144}
\right)


In [140]:
replace(str(f5[5].factor()))

'95*k**2*(k**2*y[0]**2 + y[2]**2)*(16*k**4*y[0]**4*y[1]**2 - 44*k**4*y[0]**2*y[1]**4 + k**4*y[1]**6 + 15*k**2*y[0]**4*y[2]**2 - 209*k**2*y[0]**2*y[1]**2*y[2]**2 + 44*k**2*y[1]**4*y[2]**2 - 15*y[0]**2*y[2]**4 + 16*y[1]**2*y[2]**4)/144'

### Adams–Moulton method 5 order

In [141]:
set_clip(7, 6, Rational(0, 1))
m = AdamsMoulton5(g, Matrix([p(t), q(t), r(t)]))

In [142]:
F1 = clip((T(p(t+4*h))-T(p(t+3*h)))/h - T(m[0]))
prn(F1, k)
prnlatex(F1, k)

h^0 =>


p_{t} - q*r

h^1 =>


-7*(-p_{tt} + q*r_{t} + q_{t}*r)/2

h^2 =>


...

h^4 =>


-781*(-p_{ttttt} + q*r_{tttt} + q_{tttt}*r + 4*q_{ttt}*r_{t} + 6*q_{tt}*r_{tt} + 4*q_{t}*r_{ttt})/120

h^5 =>


-(-6734*p_{tttttt} + 6761*q*r_{ttttt} + 6761*q_{ttttt}*r + 33805*q_{tttt}*r_{t} + 67610*q_{ttt}*r_{tt} + 67610*q_{tt}*r_{ttt} + 33805*q_{t}*r_{tttt})/1440

p_{t} - q r
+h\left(
- \frac{7 \left(- p_{tt} + q r_{t} + q_{t} r\right)}{2}
\right)
+h^2\left(
- \frac{37 \left(- p_{ttt} + q r_{tt} + q_{tt} r + 2 q_{t} r_{t}\right)}{6}
\right)
+h^3\left(
- \frac{175 \left(- p_{tttt} + q r_{ttt} + q_{ttt} r + 3 q_{tt} r_{t} + 3 q_{t} r_{tt}\right)}{24}
\right)
+h^4\left(
- \frac{781 \left(- p_{ttttt} + q r_{tttt} + q_{tttt} r + 4 q_{ttt} r_{t} + 6 q_{tt} r_{tt} + 4 q_{t} r_{ttt}\right)}{120}
\right)
+h^5\left(
- \frac{- 6734 p_{tttttt} + 6761 q r_{ttttt} + 6761 q_{ttttt} r + 33805 q_{tttt} r_{t} + 67610 q_{ttt} r_{tt} + 67610 q_{tt} r_{ttt} + 33805 q_{t} r_{tttt}}{1440}
\right)


In [143]:
F2 = clip((T(q(t+4*h))-T(q(t+3*h)))/h - T(m[1]))
prn(F2, k)
prnlatex(F2, k)

h^0 =>


p*r + q_{t}

h^1 =>


7*(p*r_{t} + p_{t}*r + q_{tt})/2

h^2 =>


...

h^4 =>


781*(p*r_{tttt} + p_{tttt}*r + 4*p_{ttt}*r_{t} + 6*p_{tt}*r_{tt} + 4*p_{t}*r_{ttt} + q_{ttttt})/120

h^5 =>


(6761*p*r_{ttttt} + 6761*p_{ttttt}*r + 33805*p_{tttt}*r_{t} + 67610*p_{ttt}*r_{tt} + 67610*p_{tt}*r_{ttt} + 33805*p_{t}*r_{tttt} + 6734*q_{tttttt})/1440

p r + q_{t}
+h\left(
\frac{7 \left(p r_{t} + p_{t} r + q_{tt}\right)}{2}
\right)
+h^2\left(
\frac{37 \left(p r_{tt} + p_{tt} r + 2 p_{t} r_{t} + q_{ttt}\right)}{6}
\right)
+h^3\left(
\frac{175 \left(p r_{ttt} + p_{ttt} r + 3 p_{tt} r_{t} + 3 p_{t} r_{tt} + q_{tttt}\right)}{24}
\right)
+h^4\left(
\frac{781 \left(p r_{tttt} + p_{tttt} r + 4 p_{ttt} r_{t} + 6 p_{tt} r_{tt} + 4 p_{t} r_{ttt} + q_{ttttt}\right)}{120}
\right)
+h^5\left(
\frac{6761 p r_{ttttt} + 6761 p_{ttttt} r + 33805 p_{tttt} r_{t} + 67610 p_{ttt} r_{tt} + 67610 p_{tt} r_{ttt} + 33805 p_{t} r_{tttt} + 6734 q_{tttttt}}{1440}
\right)


In [144]:
F3 = clip((T(r(t+4*h))-T(r(t+3*h)))/h - T(m[2]))
prn(F3, k)
prnlatex(F3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


7*k**2*(p*q_{t} + p_{t}*q)/2 + 7*r_{tt}/2

h^2 =>


...

h^4 =>


781*k**2*(p*q_{tttt} + p_{tttt}*q + 4*p_{ttt}*q_{t} + 6*p_{tt}*q_{tt} + 4*p_{t}*q_{ttt})/120 + 781*r_{ttttt}/120

h^5 =>


6761*k**2*(p*q_{ttttt} + p_{ttttt}*q + 5*p_{tttt}*q_{t} + 10*p_{ttt}*q_{tt} + 10*p_{tt}*q_{ttt} + 5*p_{t}*q_{tttt})/1440 + 3367*r_{tttttt}/720

k^{2} p q + r_{t}
+h\left(
\frac{7 k^{2} \left(p q_{t} + p_{t} q\right)}{2} + \frac{7 r_{tt}}{2}
\right)
+h^2\left(
\frac{37 k^{2} \left(p q_{tt} + p_{tt} q + 2 p_{t} q_{t}\right)}{6} + \frac{37 r_{ttt}}{6}
\right)
+h^3\left(
\frac{175 k^{2} \left(p q_{ttt} + p_{ttt} q + 3 p_{tt} q_{t} + 3 p_{t} q_{tt}\right)}{24} + \frac{175 r_{tttt}}{24}
\right)
+h^4\left(
\frac{781 k^{2} \left(p q_{tttt} + p_{tttt} q + 4 p_{ttt} q_{t} + 6 p_{tt} q_{tt} + 4 p_{t} q_{ttt}\right)}{120} + \frac{781 r_{ttttt}}{120}
\right)
+h^5\left(
\frac{6761 k^{2} \left(p q_{ttttt} + p_{ttttt} q + 5 p_{tttt} q_{t} + 10 p_{ttt} q_{tt} + 10 p_{tt} q_{ttt} + 5 p_{t} q_{tttt}\right)}{1440} + \frac{3367 r_{tttttt}}{720}
\right)


In [145]:
f1 = NF(F1, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f1, k)
prnlatex(f1, k)

h^0 =>


p_{t} - q*r

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


3*k**6*p*q**2*(4*p**2 - 6*p*q - q**2)*(4*p**2 + 6*p*q - q**2)/160 + 3*k**4*p*r**2*(16*p**4 - 328*p**2*q**2 + 135*q**4)/160 - 3*k**2*p*r**4*(44*p**2 - 135*q**2)/160 + 3*p*r**6/160

p_{t} - q r
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
\frac{3 k^{6} p q^{2} \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{160} + \frac{3 k^{4} p r^{2} \left(16 p^{4} - 328 p^{2} q^{2} + 135 q^{4}\right)}{160} - \frac{3 k^{2} p r^{4} \left(44 p^{2} - 135 q^{2}\right)}{160} + \frac{3 p r^{6}}{160}
\right)


In [146]:
f2 = NF(F2, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f2, k)
prnlatex(f2, k)

h^0 =>


p*r + q_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-3*k**6*p**2*q*(p**2 - 6*p*q - 4*q**2)*(p**2 + 6*p*q - 4*q**2)/160 + 3*k**4*q*r**2*(135*p**4 - 328*p**2*q**2 + 16*q**4)/160 - 3*k**2*q*r**4*(135*p**2 - 44*q**2)/160 + 3*q*r**6/160

p r + q_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{3 k^{6} p^{2} q \left(p^{2} - 6 p q - 4 q^{2}\right) \left(p^{2} + 6 p q - 4 q^{2}\right)}{160} + \frac{3 k^{4} q r^{2} \left(135 p^{4} - 328 p^{2} q^{2} + 16 q^{4}\right)}{160} - \frac{3 k^{2} q r^{4} \left(135 p^{2} - 44 q^{2}\right)}{160} + \frac{3 q r^{6}}{160}
\right)


In [147]:
f3 = NF(F3, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [F1, F2, F3], head=False)
prn(f3, k)
prnlatex(f3, k)

h^0 =>


k**2*p*q + r_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-3*k**6*r*(p - q)*(p + q)*(p**4 - 134*p**2*q**2 + q**4)/160 + 3*k**4*r**3*(11*p**4 - 82*p**2*q**2 + 11*q**4)/40 - 3*k**2*r**5*(p - q)*(p + q)/10

k^{2} p q + r_{t}
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{3 k^{6} r \left(p - q\right) \left(p + q\right) \left(p^{4} - 134 p^{2} q^{2} + q^{4}\right)}{160} + \frac{3 k^{4} r^{3} \left(11 p^{4} - 82 p^{2} q^{2} + 11 q^{4}\right)}{40} - \frac{3 k^{2} r^{5} \left(p - q\right) \left(p + q\right)}{10}
\right)


In [148]:
F4 = clip(((T(p(t+h))**2 + T(q(t+h))**2) -\
           (T(p(t))**2 + T(q(t))**2))/h)
prn(F4, k)
prnlatex(F4, k)

h^0 =>


2*(p*p_{t} + q*q_{t})

h^1 =>


p*p_{tt} + p_{t}**2 + q*q_{tt} + q_{t}**2

h^2 =>


...

h^4 =>


(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt} + q*q_{ttttt} + 5*q_{tttt}*q_{t} + 10*q_{ttt}*q_{tt})/60

h^5 =>


(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2 + q*q_{tttttt} + 6*q_{ttttt}*q_{t} + 15*q_{tttt}*q_{tt} + 10*q_{ttt}**2)/360

2 \left(p p_{t} + q q_{t}\right)
+h\left(
p p_{tt} + p_{t}^{2} + q q_{tt} + q_{t}^{2}
\right)
+h^2\left(
\frac{p p_{ttt} + 3 p_{tt} p_{t} + q q_{ttt} + 3 q_{tt} q_{t}}{3}
\right)
+h^3\left(
\frac{p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2} + q q_{tttt} + 4 q_{ttt} q_{t} + 3 q_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt} + q q_{ttttt} + 5 q_{tttt} q_{t} + 10 q_{ttt} q_{tt}}{60}
\right)
+h^5\left(
\frac{p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2} + q q_{tttttt} + 6 q_{ttttt} q_{t} + 15 q_{tttt} q_{tt} + 10 q_{ttt}^{2}}{360}
\right)


In [149]:
f4 = NF(F4, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f4, k)
prnlatex(f4, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-9*k**6*p**2*q**2*(p - q)*(p + q)*(p**2 + q**2)/16 - 3*k**4*r**2*(p**2 + q**2)*(16*p**4 - 209*p**2*q**2 + 16*q**4)/80 + 33*k**2*r**4*(p - q)*(p + q)*(p**2 + q**2)/20 - 3*r**6*(p**2 + q**2)/80

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{9 k^{6} p^{2} q^{2} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{16} - \frac{3 k^{4} r^{2} \left(p^{2} + q^{2}\right) \left(16 p^{4} - 209 p^{2} q^{2} + 16 q^{4}\right)}{80} + \frac{33 k^{2} r^{4} \left(p - q\right) \left(p + q\right) \left(p^{2} + q^{2}\right)}{20} - \frac{3 r^{6} \left(p^{2} + q^{2}\right)}{80}
\right)


In [150]:
replace(str(f4[5].collect(k)))

'k**6*(-9*y[0]**6*y[1]**2/16 + 9*y[0]**2*y[1]**6/16) + k**4*(-3*y[0]**6*y[2]**2/5 + 579*y[0]**4*y[1]**2*y[2]**2/80 + 579*y[0]**2*y[1]**4*y[2]**2/80 - 3*y[1]**6*y[2]**2/5) + k**2*(33*y[0]**4*y[2]**4/20 - 33*y[1]**4*y[2]**4/20) - 3*y[0]**2*y[2]**6/80 - 3*y[1]**2*y[2]**6/80'

In [151]:
F5 = clip(((k**2*T(p(t+h))**2 + T(r(t+h))**2) -\
           (k**2*T(p(t))**2 + T(r(t))**2))/h)
prn(F5, k)
prnlatex(F5, k)

h^0 =>


2*k**2*p*p_{t} + 2*r*r_{t}

h^1 =>


k**2*(p*p_{tt} + p_{t}**2) + r*r_{tt} + r_{t}**2

h^2 =>


...

h^4 =>


k**2*(p*p_{ttttt} + 5*p_{tttt}*p_{t} + 10*p_{ttt}*p_{tt})/60 + (r*r_{ttttt} + 5*r_{tttt}*r_{t} + 10*r_{ttt}*r_{tt})/60

h^5 =>


k**2*(p*p_{tttttt} + 6*p_{ttttt}*p_{t} + 15*p_{tttt}*p_{tt} + 10*p_{ttt}**2)/360 + (r*r_{tttttt} + 6*r_{ttttt}*r_{t} + 15*r_{tttt}*r_{tt} + 10*r_{ttt}**2)/360

2 k^{2} p p_{t} + 2 r r_{t}
+h\left(
k^{2} \left(p p_{tt} + p_{t}^{2}\right) + r r_{tt} + r_{t}^{2}
\right)
+h^2\left(
\frac{k^{2} \left(p p_{ttt} + 3 p_{tt} p_{t}\right)}{3} + \frac{r r_{ttt} + 3 r_{tt} r_{t}}{3}
\right)
+h^3\left(
\frac{k^{2} \left(p p_{tttt} + 4 p_{ttt} p_{t} + 3 p_{tt}^{2}\right)}{12} + \frac{r r_{tttt} + 4 r_{ttt} r_{t} + 3 r_{tt}^{2}}{12}
\right)
+h^4\left(
\frac{k^{2} \left(p p_{ttttt} + 5 p_{tttt} p_{t} + 10 p_{ttt} p_{tt}\right)}{60} + \frac{r r_{ttttt} + 5 r_{tttt} r_{t} + 10 r_{ttt} r_{tt}}{60}
\right)
+h^5\left(
\frac{k^{2} \left(p p_{tttttt} + 6 p_{ttttt} p_{t} + 15 p_{tttt} p_{tt} + 10 p_{ttt}^{2}\right)}{360} + \frac{r r_{tttttt} + 6 r_{ttttt} r_{t} + 15 r_{tttt} r_{tt} + 10 r_{ttt}^{2}}{360}
\right)


In [152]:
f5 = NF(F5, [p(t).diff(t), q(t).diff(t), r(t).diff(t)], [f1, f2, f3], head=True)
prn(f5, k)
prnlatex(f5, k)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-3*k**8*p**2*q**2*(4*p**2 - 6*p*q - q**2)*(4*p**2 + 6*p*q - q**2)/80 - 3*k**6*r**2*(15*p**6 - 193*p**4*q**2 + q**6)/80 + 3*k**4*q**2*r**4*(193*p**2 - 44*q**2)/80 + 3*k**2*r**6*(15*p**2 - 16*q**2)/80

0
+h\left(
0
\right)
+h^2\left(
0
\right)
+h^3\left(
0
\right)
+h^4\left(
0
\right)
+h^5\left(
- \frac{3 k^{8} p^{2} q^{2} \left(4 p^{2} - 6 p q - q^{2}\right) \left(4 p^{2} + 6 p q - q^{2}\right)}{80} - \frac{3 k^{6} r^{2} \left(15 p^{6} - 193 p^{4} q^{2} + q^{6}\right)}{80} + \frac{3 k^{4} q^{2} r^{4} \left(193 p^{2} - 44 q^{2}\right)}{80} + \frac{3 k^{2} r^{6} \left(15 p^{2} - 16 q^{2}\right)}{80}
\right)


In [153]:
replace(str(f5[5].factor()))

'-3*k**2*(k**2*y[0]**2 + y[2]**2)*(16*k**4*y[0]**4*y[1]**2 - 44*k**4*y[0]**2*y[1]**4 + k**4*y[1]**6 + 15*k**2*y[0]**4*y[2]**2 - 209*k**2*y[0]**2*y[1]**2*y[2]**2 + 44*k**2*y[1]**4*y[2]**2 - 15*y[0]**2*y[2]**4 + 16*y[1]**2*y[2]**4)/80'

In [154]:
end = time.time()
print(end - start)

3310.569954395294
